# 1 Descargar Ventas y Visitas (aje-prd-analytics-artifacts-s3)

In [1]:
# import psutil
# import time
# import threading
# import matplotlib.pyplot as plt

# # Variables globales
# mem_usage_gb = []
# mem_usage_percent = []
# cpu_usage_gb = []
# cpu_usage_percent = []
# timestamps = []
# running = True  # Variable de control

# # Función de monitoreo en segundo plano
# def monitor_resources():
#     global running
#     start_time = time.time()
    
#     while running:
#         # Obtener información del sistema
#         mem_info = psutil.virtual_memory()
#         cpu_percent = psutil.cpu_percent(interval=0.5)  # Medir CPU sin bloquear
#         cpu_used_gb = (cpu_percent / 100) * (psutil.virtual_memory().total / (1024**3))  # Estimación del uso de CPU en GB
        
#         # Guardar datos
#         timestamp = time.time() - start_time
#         mem_usage_gb.append(mem_info.used / (1024 ** 3))  # RAM usada en GB
#         mem_usage_percent.append(mem_info.percent)  # RAM en %
#         cpu_usage_gb.append(cpu_used_gb)  # CPU en GB
#         cpu_usage_percent.append(cpu_percent)  # CPU en %
#         timestamps.append(timestamp)
        
#         # Imprimir valores en tiempo real
#         # print(f"Tiempo: {timestamp:.2f}s | RAM: {mem_info.percent:.2f}% ({mem_usage_gb[-1]:.2f} GB) | CPU: {cpu_percent:.2f}% ({cpu_used_gb:.2f} GB)")
        
#         time.sleep(1)  # Medir cada 1 segundo

# # Iniciar monitoreo en segundo plano
# monitor_thread = threading.Thread(target=monitor_resources, daemon=True)
# monitor_thread.start()

# print("Monitoreo de CPU y RAM iniciado en segundo plano...")

In [2]:
!pip install 'awswrangler[redshift]'
!pip install psycopg2-binary
!pip install openpyxl
!pip install redshift-connector

In [3]:
import pandas as pd
import numpy as np
import redshift_connector
import awswrangler as wr
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os
import boto3
import io
import pytz
import re

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
def comprobar_inputs():
    # Conectarse a S3
    s3 = boto3.client("s3")
    bucket_name = "aje-prd-analytics-artifacts-s3"
    prefix = "pedido_sugerido/data-v1/mexico/"
    
    hoy = datetime.now(pytz.timezone("America/Lima")).date()
    errores = []

    # Listar objetos en la ruta de S3
    objetos = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    if "Contents" not in objetos:
        print("ERROR: No se encontraron archivos en la ruta especificada.")
        return

    # Iterar sobre los objetos
    for objeto in objetos["Contents"]:
        key = objeto["Key"]
        print(key)
        # Omitir "carpetas" en S3
        if key.endswith("/"):
            continue

        last_modified = objeto["LastModified"].date()
        size_kb = objeto["Size"] / 1024  # Convertir a KB

        # Verificar si el objeto tiene contenido real (FULL_OBJECT)
        if objeto["Size"] == 0:
            errores.append(f"ERROR: El archivo {key} está vacío.")

        # Verificar si el archivo ha sido modificado hoy
        if last_modified != hoy:
            errores.append(f"ERROR: El archivo {key} no ha sido modificado hoy ({hoy}), su ultima fecha de modificacion fue {last_modified}.")

        # Verificar si el tamaño del archivo es menor a 1 KB
        if size_kb < 1:
            errores.append(f"ALERTA: El archivo {key} tiene un tamaño menor a 1 KB ({size_kb:.2f} KB).")

    # Mostrar los errores y lanzar una excepción si es necesario
    if errores:
        for error in errores:
            print(error)
        raise ValueError("Se encontraron problemas con los archivos en S3.")
    else:
        print("Todo bien :D")

In [5]:
comprobar_inputs()

pedido_sugerido/data-v1/mexico/
pedido_sugerido/data-v1/mexico/maestro_productos_mexico000
pedido_sugerido/data-v1/mexico/ventas_mexico000
pedido_sugerido/data-v1/mexico/visitas_mexico000
Todo bien :D


## 1.0 Descargar Maestro de productos

In [6]:
# PARA 2024
query = f"""select * from 
    comercial_mexico.dim_producto
    where estado='A' and instancia='MX';
    """

con = wr.data_api.redshift.connect(
    cluster_id="dwh-cloud-storage-salesforce-prod",
    database="dwh_prod",
    db_user="dwhuser",
)
maestro_prod = wr.data_api.rds.read_sql_query(query, con)

In [7]:
maestro_prod.head()

,id_producto,instancia,desc_compania,cod_articulo_magic,desc_articulo,desc_articulo_corto,cod_linea,desc_linea,cod_familia,desc_familia,...,flg_tipo_composicion,flg_linea,paquete,flg_preventa,flgabastec,flgskuplan,flg_explosion,fecha_creacion,cod_unidad_negocio,desc_unidad_negocio
0,MX|0030|26152,MX,AJEMEX ...,26152,BASE DE BEBIDA NECTAR DURAZNO BN-MX-70008/2U,BASE DE BEBIDA NECTAR DURAZNO,03,PRODUCTO INTERMEDIO,005,BASE DE BEBIDA PT,...,C,Te,1,N,OP,N,S,None,,None
1,MX|0030|26476,MX,AJEMEX ...,26476,BASE LIQUIDA PARTE 2 (COLA) PT,BASE LIQUIDA PARTE 2 (COLA) PT,03,PRODUCTO INTERMEDIO,005,BASE DE BEBIDA PT,...,C,Te,1,N,OP,N,S,2008-02-07,,None
2,MX|0030|27119,MX,AJEMEX ...,27119,BOTELLA SOPLADA-ETIQUETADA DE FIRST NARANJA 62...,BOTELLA SOPLADA-ETIQUETADA DE,03,PRODUCTO INTERMEDIO,006,BOTELLAS SOPLADAS,...,C,Te,1,N,OP,N,S,2008-04-14,,None
3,MX|0030|28061,MX,AJEMEX ...,28061,BASE DE BEBIDA GASIFICADA FIRST TORONJA BG-MX-...,BASE DE BEBIDA GASIFICADA FIRS,03,PRODUCTO INTERMEDIO,005,BASE DE BEBIDA PT,...,C,Te,1,N,OP,N,S,2008-06-20,,None
4,MX|0030|29819,MX,AJEMEX ...,29819,JARABE TERMINADO BIG COUNTRY TETRA DURAZNO,JARABE TERMINADO BIG COUNTRY T,03,PRODUCTO INTERMEDIO,003,JARABE TERMINADO,...,C,Te,1,N,OP,N,S,2008-11-27,,None


In [8]:
maestro_prod[["cod_articulo_magic", "desc_articulo"]].drop_duplicates().reset_index(
    drop=True
).to_csv("Input/MX_maestro_productos.csv", index=False)

In [9]:
rutas_test = [1155,1158,1074,1065]
rutas_test

[1155, 1158, 1074, 1065]

In [10]:
len(rutas_test)

4

In [11]:
clientes_ruta_test = []

In [12]:
def descargar_visitas():
    # Guardar clientes de las rutas
    global clientes_ruta_test

    # Conectarse a S3
    s3 = boto3.client("s3")
    bucket_name = "aje-prd-analytics-artifacts-s3"
    prefix = "pedido_sugerido/data-v1/mexico/"

    # Listar objetos en la ruta de S3
    objetos = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    print(objetos)

    # Iterar sobre los objetos
    for objeto in objetos["Contents"]:
        if objeto["Size"] > 0 and objeto["Key"].split("/")[-1] == "visitas_mexico000":
            # Obtener el nombre del objeto y descargarlo
            nombre_archivo = objeto["Key"].split("/")[-1]
            # Descargar el archivo en memoria
            response = s3.get_object(Bucket=bucket_name, Key=objeto["Key"])
            content = response["Body"].read()
            # Filtrar y guardar el archivo si corresponde
            if nombre_archivo == "visitas_mexico000":
                # Convertir los bytes a DataFrame de Pandas
                df = pd.read_csv(io.BytesIO(content), sep=";")
                df = df[
                    (df["compania__c"] == 30)
                    & (df["cod_ruta"].isin(rutas_test))
                    & (df.codigo_canal__c == 2)
                ].reset_index(drop=True)
                # Guardamos estos clientes para descargarlos en ventas (por si cambiaron de ruta)
                clientes_ruta_test = df["codigo_cliente__c"].unique()
                nombre_csv = f"Input/{nombre_archivo}.parquet"
                df.to_parquet(nombre_csv,index=False)
            else:
                continue

In [13]:
%%time
descargar_visitas()

{'ResponseMetadata': {'RequestId': 'S2M5V9QRYC63YHAE', 'HostId': 'uXjCkAH9qf/e1Z/8YG7cPqrHeFMif6ZE5WxFwPY4HEkt3XkSI/7YnGnTD/WjWddhlafMI5iHzPppwZyc6U5Yh6jMzOPbic3kfLknmqlnucY=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'uXjCkAH9qf/e1Z/8YG7cPqrHeFMif6ZE5WxFwPY4HEkt3XkSI/7YnGnTD/WjWddhlafMI5iHzPppwZyc6U5Yh6jMzOPbic3kfLknmqlnucY=', 'x-amz-request-id': 'S2M5V9QRYC63YHAE', 'date': 'Mon, 07 Jul 2025 17:57:39 GMT', 'x-amz-bucket-region': 'us-east-2', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'pedido_sugerido/data-v1/mexico/', 'LastModified': datetime.datetime(2024, 2, 8, 8, 32, 11, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD'}, {'Key': 'pedido_sugerido/data-v1/mexico/maestro_productos_mexico000', 'LastModified': datetime.datetime(2025, 7, 7, 15, 43, 15, tzinfo=tzlocal()), 'ETag': '"33e338c32af37162f510716a33a5b6f6-

In [14]:
def descargar_ventas():
    # Conectarse a S3
    s3 = boto3.client("s3")
    bucket_name = "aje-prd-analytics-artifacts-s3"
    prefix = "pedido_sugerido/data-v1/mexico/"

    # Listar objetos en la ruta de S3
    objetos = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    print(objetos)

    # Iterar sobre los objetos
    for objeto in objetos["Contents"]:
        if objeto["Size"] > 0 and objeto["Key"].split("/")[-1] != "visitas_mexico000":
            # Obtener el nombre del objeto y descargarlo
            nombre_archivo = objeto["Key"].split("/")[-1]

            # Descargar el archivo en memoria
            response = s3.get_object(Bucket=bucket_name, Key=objeto["Key"])
            content = response["Body"].read()

            # Filtrar y guardar el archivo si corresponde
            if nombre_archivo == "ventas_mexico000":
                # Convertir los bytes a DataFrame de Pandas
                df = pd.read_csv(io.BytesIO(content), sep=";")
                df = df[
                    (df["cod_compania"] == 30)
                    & (
                        (df["cod_ruta"].isin(rutas_test))
                        | (df["cod_cliente"].isin(clientes_ruta_test))
                    )
                ].reset_index(drop=True)[['id_cliente', 'id_sucursal', 'id_producto',
       'fecha_liquidacion',"cod_ruta","cod_modulo",
       'cod_zona', 'cant_cajafisicavta', 'cant_cajaunitvta','imp_netovta',
       'cod_compania', 'desc_compania', 'cod_sucursal',
       'desc_sucursal', 'cod_pais', 'fecha_creacion_cliente', 'cod_cliente',
       'desc_marca', 'desc_formato', 'desc_categoria', 'cod_giro',
       'cod_subgiro', 'desc_giro', 'desc_subgiro', 'fecha_proceso']]
                nombre_csv = f"Input/{nombre_archivo}.parquet"
                df.to_parquet(nombre_csv,index=False)
            else:
                continue

In [15]:
%%time
descargar_ventas()

{'ResponseMetadata': {'RequestId': 'PD853XT3XDT2HGWW', 'HostId': '3TLMzJPw5wJCWfEW7R+RFMVhr0aJaNOY/zOxxYCikESo8do0qtHlyoFvjGYJs3nO2t/Xz162cHngr1OlICsn8A==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '3TLMzJPw5wJCWfEW7R+RFMVhr0aJaNOY/zOxxYCikESo8do0qtHlyoFvjGYJs3nO2t/Xz162cHngr1OlICsn8A==', 'x-amz-request-id': 'PD853XT3XDT2HGWW', 'date': 'Mon, 07 Jul 2025 17:57:40 GMT', 'x-amz-bucket-region': 'us-east-2', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'pedido_sugerido/data-v1/mexico/', 'LastModified': datetime.datetime(2024, 2, 8, 8, 32, 11, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD'}, {'Key': 'pedido_sugerido/data-v1/mexico/maestro_productos_mexico000', 'LastModified': datetime.datetime(2025, 7, 7, 15, 43, 15, tzinfo=tzlocal()), 'ETag': '"33e338c32af37162f510716a33a5b6f6-3"', 'ChecksumAlgorithm': ['CRC64NVME'],

In [16]:
def descargarStockDiario():
    # Obtener la fecha y año actual
    current_date = datetime.now(pytz.timezone("America/Lima"))

    # Formatear la fecha en el formato "YYYYMM"
    formatted_date = current_date.strftime("%Y%m")
    # PARA 2024
    query = f"""select * from 
        cadena_prod.fact_probabilidad_quiebre_stock
        where id_pais='MX' and id_periodo='{formatted_date}';
        """

    con = wr.data_api.redshift.connect(
        cluster_id="dwh-cloud-storage-salesforce-prod",
        database="dwh_prod",
        db_user="dwhuser",
    )
    df_quiebres = wr.data_api.rds.read_sql_query(query, con)
    return df_quiebres

In [17]:
df_quiebres = descargarStockDiario()

In [18]:
df_quiebres

,id_pais,id_periodo,id_sucursal,id_articulo,fecha_inventario,cant_cajaunitaria,cant_cajafisica,cant_cajaunitaria_plan,cant_cajafisica_plan
0,MX,202507,0030|0070,0030|599966,2025-07-03,0.0000,0.0000,0.0000,0.0000
1,MX,202507,0030|0070,0030|522661,2025-07-05,0.0000,0.0000,0.0000,0.0000
2,MX,202507,0030|0070,0030|598901,2025-07-05,615.0900,1015.0000,109.6971,181.0184
3,MX,202507,0030|0070,0030|523595,2025-07-05,453.6000,4536.0000,0.0000,0.0000
4,MX,202507,0030|0070,0030|517163,2025-07-04,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...
19168,MX,202507,0030|84,0030|517297,2025-07-03,70.7608,374.0000,13.1522,69.5148
19169,MX,202507,0030|84,0030|518501,2025-07-05,0.0000,0.0000,0.0000,0.0000
19170,MX,202507,0030|18,0030|517297,2025-07-04,64.5172,341.0000,3.7451,19.7948
19171,MX,202507,0030|18,0030|511053,2025-07-03,0.0000,0.0000,0.0000,0.0000


## 1.1 Juntar VENTAS y VISITAS

In [19]:
pan_ventas = pd.read_parquet("Input/ventas_mexico000.parquet")
pan_visitas = pd.read_parquet("Input/visitas_mexico000.parquet")

In [20]:
pan_ventas["cod_articulo_magic"] = pan_ventas["id_producto"].str.split("|").str[-1]
pan_ventas["cod_articulo_magic"] = pan_ventas["cod_articulo_magic"].astype(int)

In [21]:
pan_ventas.shape

(53159, 26)

In [22]:
# Nos quedamos solo con las filas del proceso de hoy
# Obtener la fecha de hoy en el mismo formato
hoy = int(datetime.now(pytz.timezone("America/Lima")).strftime('%Y%m%d'))
print(hoy)
# Filtrar solo las filas con la fecha de hoy
pan_ventas = pan_ventas[pan_ventas['fecha_proceso'] == hoy]

20250707


In [23]:
pan_ventas.shape

(53159, 26)

In [24]:
# Obtener la fecha y año actual
current_date = datetime.now(pytz.timezone("America/Lima"))
# Formatear la fecha en el formato "YYYYMM"
formatted_date = current_date.strftime("%Y-%m-%d")
formatted_date

'2025-07-07'

In [25]:
# Establecer la conexión con S3
bucket_name = 'aje-analytics-ps-backup'  # nombre de bucket en S3
file_name = f'PS_Mexico/Input/visitas_mexico000_{formatted_date}.csv'  # nombre para el archivo en S3
s3_path = f's3://{bucket_name}/{file_name}'

# Escribir el dataframe en S3 con AWS Data Wrangler
wr.s3.to_csv(pan_visitas, s3_path, index=False)

{'paths': ['s3://aje-analytics-ps-backup/PS_Mexico/Input/visitas_mexico000_2025-07-07.csv'],
 'partitions_values': {}}

In [26]:
pan_visitas = pan_visitas.rename(columns={'sucursal__c': 'cod_sucursal'})

In [27]:
pan_ventas.cod_articulo_magic.nunique()

73

In [28]:
# PAN VENTAS
pan_ventas["cod_compania"] = (
    pan_ventas["cod_compania"].astype(str).apply(lambda x: str(int(x)).rjust(4, "0"))
)
pan_ventas["id_cliente"] = (
    "MX"
    + "|"
    + pan_ventas["cod_compania"].astype(str)
    + "|"
    + pan_ventas["cod_cliente"].astype(str)
)

# PAN VISITAS
pan_visitas["compania__c"] = (
    pan_visitas["compania__c"].astype(str).apply(lambda x: str(int(x)).rjust(4, "0"))
)
pan_visitas["id_cliente"] = (
    "MX"
    + "|"
    + pan_visitas["compania__c"].astype(str)
    + "|"
    + pan_visitas["codigo_cliente__c"].astype(str)
)

In [29]:
pan_ventas.groupby("cod_sucursal")["id_cliente"].nunique()

cod_sucursal
84     903
112    567
113    130
Name: id_cliente, dtype: int64

In [30]:
pan_ventas.cod_articulo_magic.nunique()

73

In [31]:
pan_ventas["fecha_liquidacion"].sort_values().unique()[-10:]

array(['2025-06-25', '2025-06-26', '2025-06-27', '2025-06-28',
       '2025-06-30', '2025-07-01', '2025-07-02', '2025-07-03',
       '2025-07-04', '2025-07-05'], dtype=object)

In [32]:
pan_visitas["ultima_visita"].sort_values().unique()[-10:]

array(['2025-06-30', '2025-07-01', '2025-07-02', '2025-07-03',
       '2025-07-04', '2025-07-05'], dtype=object)

In [33]:
pan_ventas["id_cliente"].nunique()

1472

In [34]:
pan_visitas["id_cliente"].nunique()

1462

In [35]:
# Eliminar Duplicados y quedarse con la ultima visita más reciente
visita_default_semana_atras = (datetime.now(pytz.timezone("America/Lima")) - timedelta(days=7)).strftime("%Y-%m-%d")
pan_visitas["ultima_visita"] = pan_visitas["ultima_visita"].fillna(
    visita_default_semana_atras
)
pan_visitas = (
    pan_visitas.sort_values(["id_cliente", "ultima_visita"], ascending=False)
    .groupby("id_cliente")
    .head(1)
)

In [36]:
# Combina ventas y visitas (nos quedamos con ruta y módulo de visitas si es nulo, usamos el valor de ventas)
df_merged = pd.merge(
    pan_ventas,
    pan_visitas[
        [
            "id_cliente",
            "dias_de_visita__c",
            "periodo_de_visita__c",
            "ultima_visita",
            "cod_ruta",
            "cod_modulo",
            "cod_sucursal",
        ]
    ],
    on="id_cliente",
    # how='outer',  # Este es un merge 'outer', para conservar todos los clientes de ambos DataFrames.
    how="inner",
    suffixes=(
        "_df1",
        "_df2",
    ),  # Añadimos sufijos para diferenciar las columnas que se solapan
)

# Reemplazar los valores nulos de 'cod_ruta_df2' con los valores de 'cod_ruta_df1', y viceversa
# Si 'cod_ruta_df2' es nulo, se usará el valor de 'cod_ruta_df1'. Si ambos son nulos, se mantendrá nulo.
df_merged["cod_ruta"] = df_merged["cod_ruta_df2"].combine_first(
    df_merged["cod_ruta_df1"]
)
df_merged["cod_modulo"] = df_merged["cod_modulo_df2"].combine_first(
    df_merged["cod_modulo_df1"]
)
df_merged["cod_sucursal"] = df_merged["cod_sucursal_df2"].combine_first(
    df_merged["cod_sucursal_df1"]
)

# Eliminar las columnas innecesarias ('cod_ruta_df1', 'cod_ruta_df2', 'cod_modulo_df1', 'cod_modulo_df2')
df_final = df_merged.drop(
    columns=["cod_ruta_df1", "cod_ruta_df2", "cod_modulo_df1", "cod_modulo_df2", "cod_sucursal_df1", "cod_sucursal_df2"]
)

In [37]:
print(df_final['id_cliente'])

0        MX|0030|1708421
1        MX|0030|1708421
2        MX|0030|1705944
3        MX|0030|1599470
4        MX|0030|1595256
              ...       
51694    MX|0030|1513791
51695    MX|0030|1764956
51696    MX|0030|1601113
51697    MX|0030|1630634
51698    MX|0030|1513791
Name: id_cliente, Length: 51699, dtype: object


In [38]:
pan_ventas = df_final.copy()

In [39]:
pan_ventas["cod_cliente"] = pan_ventas["cod_cliente"].astype(str)
pan_ventas["cod_ruta"] = pan_ventas["cod_ruta"].astype(int)
pan_ventas["cod_modulo"] = pan_ventas["cod_modulo"].astype(int)

In [40]:
pan_ventas.cod_ruta.unique()

array([1158, 1155, 1065, 1074])

In [41]:
# Al cruzar visita con ventas. hay clientes de. visita que no tienen informacion de venta, esto se remplaza a nivel de ruta.
# si la ruta entera no tiene ventas, se ignora
final_pan_ventas = pd.DataFrame()
# Este df guarda clientes con visitas y sin ventas para luego recomendarlos por separado
df_clientes_con_visitas_sin_ventas = pd.DataFrame()
for ruta in pan_ventas["cod_ruta"].unique():
    temp_df = pan_ventas[pan_ventas["cod_ruta"] == ruta]
    # Si la ruta entera no tiene ninguna venta, se guarda en un dataframe aparte
    if len(temp_df) == (temp_df["cod_articulo_magic"].isnull().sum()):
        df_clientes_con_visitas_sin_ventas = pd.concat(
            [df_clientes_con_visitas_sin_ventas, temp_df], axis=0
        ).reset_index(drop=True)

In [42]:
pan_ventas["id_cliente"].nunique()

1287

In [43]:
pan_ventas.groupby("cod_sucursal").agg(
    {"cod_articulo_magic": "nunique", "id_cliente": "nunique", "cod_ruta": "nunique"}
)

,cod_articulo_magic,id_cliente,cod_ruta
cod_sucursal,,,
84,70,726,2
112,72,561,2


In [44]:
pan_ventas.to_parquet("Processed/ventas_mexico_12m.parquet",index=False)

# 2 Ajustar DF de clientes: Se filtrará a los clientes que tienen visita al día siguiente

## 2.2 Combinar Ventas con Subgiro, Descripcion de productos, Segmento del cliente y Fecha de creacion

In [45]:
import pandas as pd

# Leer Archivos
pan_ventas = pd.read_parquet("Processed/ventas_mexico_12m.parquet")

# CAMBIAR DEPORADE POR SPORADE
pan_ventas["desc_marca"] = pan_ventas["desc_marca"].str.strip()
pan_ventas["desc_marca"] = pan_ventas["desc_marca"].replace({"DEPORADE": "SPORADE"})

pan_prod = pd.read_csv("Input/MX_maestro_productos.csv")
df_seg = pd.read_csv("Input/D_SubSegmentacion_mexico_2024-07-25.csv")

# Crear ID_CLIENTE
# PAN_VENTAS
pan_ventas["cod_sucursal"] = pan_ventas["cod_sucursal"].astype(str)
# PAN SEGMENTOS
pan_segmentos = df_seg[["id_cliente", "new_segment"]].drop_duplicates()

In [46]:
pan_ventas = pd.merge(
    pan_ventas,
    pan_prod[["cod_articulo_magic", "desc_articulo"]],
    how="left",
    on="cod_articulo_magic",
)
# pan_ventas=pd.merge(pan_ventas,pan_giros[["id_cliente","desc_sucursal","desc_giro","desc_subgiro"]],how='left',on='id_cliente')
# pan_ventas=pd.merge(pan_ventas,pan_giros[["id_cliente","desc_sucursal"]],how='left',on='id_cliente')
pan_ventas = pd.merge(pan_ventas, pan_segmentos, on="id_cliente", how="left")
# pan_ventas=pd.merge(pan_ventas,pan_fechacreacion[["id_cliente","fecha_creacion"]],how='left',on='id_cliente')
pan_ventas["new_segment"] = pan_ventas["new_segment"].fillna("OPTIMIZAR")
pan_ventas["mes"] = pd.to_datetime(pan_ventas["fecha_liquidacion"]).dt.strftime(
    "%Y-%m-01"
)

In [47]:
pan_ventas.to_parquet("Processed/ventas_mexico_12m.parquet",index=False)

## 2.3 Seleccionando SKUs que no se commpran en los ultimos 3 dias
La variable **sucursal_sku** contiene todos los SKUs disponibles por sucursal

In [48]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [49]:
# pan_ventas = pd.read_csv(
#     "Processed/ventas_mexico_12m.csv", dtype={"dias_de_visita__c": "str"}
# )
pan_ventas = pd.read_parquet("Processed/ventas_mexico_12m.parquet")

In [50]:
sucursales = pan_ventas["cod_sucursal"].unique()
sucursales

array(['84', '112'], dtype=object)

In [51]:
# Obtener la fecha actual
fecha_actual = datetime.now(pytz.timezone("America/Lima")).date()
# Lista para almacenar las fechas
last_3_days = []
dias_atras = 3
sucursal_sku = {}
# Obtener las fechas de los últimos 3 días
for i in range(1, dias_atras + 1):
    fecha = fecha_actual - timedelta(days=i)
    fecha_formato = fecha.strftime("%Y-%m-%d")
    last_3_days.append(fecha_formato)

In [52]:
last_3_days

['2025-07-06', '2025-07-05', '2025-07-04']

In [53]:
for sucursal in sucursales:
    temp = pan_ventas[pan_ventas["cod_sucursal"] == sucursal]
    # sku_fecha=temp[["cod_articulo_magic","fecha_liquidacion"]].sort_values(["cod_articulo_magic","fecha_liquidacion"]).drop_duplicates().reset_index(drop=True)
    # sku_fecha=sku_fecha.groupby("cod_articulo_magic")[["cod_articulo_magic","fecha_liquidacion"]].tail(3)
    # #Listando SKUs comprados al menos una vez en los ultimos 3 dias
    # sku_in_last_3_days=sku_fecha[sku_fecha.fecha_liquidacion.isin(last_3_days)]["cod_articulo_magic"].unique()
    # print(f"Sucursal {sucursal} eliminando SKUs:",sku_fecha[(~sku_fecha["cod_articulo_magic"].isin(sku_in_last_3_days))].reset_index(drop=True)["cod_articulo_magic"].sort_values().unique())
    # #SKU COMPRADO AL MENOS una vez en los ultimos 3 dias
    # sku_fecha=sku_fecha[sku_fecha["cod_articulo_magic"].isin(sku_in_last_3_days)].reset_index(drop=True)
    # temp=temp[temp["cod_articulo_magic"].isin(sku_fecha["cod_articulo_magic"].unique())].reset_index(drop=True)
    sucursal_sku[sucursal] = temp["cod_articulo_magic"].sort_values().unique()

In [54]:
sucursal_sku

{'84': array([ 76643,  77400,  77401,  77411,  77412, 500171, 500264, 500330,
        500354, 509826, 515614, 516209, 517140, 517146, 517262, 517263,
        517297, 522352, 522353, 522354, 522355, 522373, 522442, 522512,
        522574, 522575, 522576, 522577, 522632, 522633, 522634, 522661,
        522662, 522713, 522721, 522722, 522723, 522795, 522856, 523125,
        523278, 523437, 523438, 523439, 523463, 523519, 523538, 523594,
        523595, 523596, 598801, 598901, 599111, 599212, 599218, 599228,
        599565, 599587, 599590, 599831, 599832, 599837, 599876, 622631,
        800054, 800058, 800060, 800069, 800070, 800071]),
 '112': array([ 76643,  77400,  77401,  77411,  77412,  77413, 500171, 500264,
        500330, 500354, 509600, 509826, 515614, 516209, 517140, 517146,
        517262, 517263, 517297, 522352, 522353, 522354, 522355, 522373,
        522442, 522512, 522574, 522575, 522576, 522577, 522632, 522633,
        522634, 522661, 522662, 522713, 522722, 522723, 522795, 5

## 2.4 Filtrar clientes a visitar mañana

In [55]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

In [56]:
pan_ventas = pd.read_parquet("Processed/ventas_mexico_12m.parquet")

In [57]:
# Filtrar ventas mayores a 0
# pan_ventas=pan_ventas[pan_ventas["imp_netovta"]>0]

# En lugar de omitir estos clientes, mejor los consideramos como OPTIMIZAR
pan_ventas.loc[pan_ventas["imp_netovta"] <= 0, "new_segment"] = "OPTIMIZAR"

In [58]:
pan_ventas.id_cliente.nunique()

1287

In [59]:
pan_ventas.dias_de_visita__c.unique()

array(['6', '2', '1', '3', '4', '5'], dtype=object)

In [60]:
pan_ventas.dias_de_visita__c.value_counts()

dias_de_visita__c
3    11065
5     9304
2     8645
1     8552
4     7770
6     7041
Name: count, dtype: int64

In [61]:
data_test = (
    pan_ventas[
        ["id_cliente", "dias_de_visita__c", "periodo_de_visita__c", "ultima_visita"]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)
data_test["ultima_visita"] = pd.to_datetime(
    data_test["ultima_visita"], format="%Y-%m-%d"
)
# Calcula la diferencia en días entre la fecha de cada fila y la fecha de hoy
fecha_actual = datetime.now() + timedelta(days=0)
data_test["dias_pasados"] = (fecha_actual - data_test["ultima_visita"]).dt.days
fecha_actual

datetime.datetime(2025, 7, 7, 17, 58, 24, 160683)

In [62]:
# Obtener el día de la semana actual (1 para lunes, 2 para martes, ..., 7 para domingo)
dia_actual = datetime.now(pytz.timezone("America/Lima")).weekday() + 1

# Si hoy es domingo (7), el día siguiente es lunes (1), de lo contrario, es el siguiente día al actual
if dia_actual == 6:
    dia_siguiente = 7
    # dia_siguiente=6
else:
    dia_siguiente = (dia_actual + 1) % 7
    # dia_siguiente = (dia_actual ) % 7

# Filtrar los clientes que serán visitados mañana
clientes_a_visitar_manana = data_test[
    data_test["dias_de_visita__c"]
    .astype(str)
    .apply(lambda x: str(dia_siguiente) in x.split(";"))
].reset_index(drop=True)
print("dia_actual", dia_actual)
print("dia_siguiente", dia_siguiente)

dia_actual 1
dia_siguiente 2


In [63]:
# Definir condiciones de filtro
condicion_f1 = clientes_a_visitar_manana["periodo_de_visita__c"] == "F1"
condicion_f2 = (clientes_a_visitar_manana["periodo_de_visita__c"] == "F2") & (
    clientes_a_visitar_manana["dias_pasados"] > 13
)
condicion_f3 = (clientes_a_visitar_manana["periodo_de_visita__c"] == "F3") & (
    clientes_a_visitar_manana["dias_pasados"] > 20
)
condicion_f4 = (clientes_a_visitar_manana["periodo_de_visita__c"] == "F4") & (
    clientes_a_visitar_manana["dias_pasados"] > 27
)

# Aplicar las condiciones de filtro
clientes_a_visitar_manana = clientes_a_visitar_manana[
    condicion_f1 | condicion_f2 | condicion_f3 | condicion_f4
].reset_index(drop=True)

In [64]:
clientes_a_visitar_manana.isnull().sum()

id_cliente              0
dias_de_visita__c       0
periodo_de_visita__c    0
ultima_visita           0
dias_pasados            0
dtype: int64

In [65]:
clientes_a_visitar_manana.periodo_de_visita__c.unique()

array(['F1'], dtype=object)

In [66]:
clientes_a_visitar_manana.dias_de_visita__c.unique()

array(['2'], dtype=object)

In [67]:
pan_ventas.groupby("cod_ruta")["dias_de_visita__c"].unique().reset_index()

,cod_ruta,dias_de_visita__c
0,1065,"[2, 3, 6, 4, 5, 1]"
1,1074,"[3, 2, 1, 4, 5, 6]"
2,1155,"[1, 3, 2, 4, 5, 6]"
3,1158,"[6, 2, 4, 3, 5, 1]"


In [68]:
pan_ventas = pan_ventas[
    pan_ventas["id_cliente"].isin(clientes_a_visitar_manana["id_cliente"])
].reset_index(drop=True)
pan_ventas

,id_cliente,id_sucursal,id_producto,fecha_liquidacion,cod_zona,cant_cajafisicavta,cant_cajaunitvta,imp_netovta,cod_compania,desc_compania,...,cod_articulo_magic,dias_de_visita__c,periodo_de_visita__c,ultima_visita,cod_ruta,cod_modulo,cod_sucursal,desc_articulo,new_segment,mes
0,MX|0030|1599470,MX|0030|84,MX|0030|516209,2024-07-25,1175,7.0,0.8750,298.4479,0030,AJEMEX,...,516209,2,F1,2025-07-01,1158,11582,84,SPORADE PONCHE DE FRUTAS PET NO RETORNABLE 625...,MANTENER,2024-07-01
1,MX|0030|1598515,MX|0030|84,MX|0030|500264,2024-07-04,1175,1.0,0.2000,80.2078,0030,AJEMEX,...,500264,2,F1,2025-07-01,1155,11552,84,BIG COLA REGULAR PET NO RETORNABLE 500 ML 12 PACK,MANTENER,2024-07-01
2,MX|0030|1598515,MX|0030|84,MX|0030|500264,2024-07-04,1175,1.0,0.2000,80.2078,0030,AJEMEX,...,500264,2,F1,2025-07-01,1155,11552,84,BIG COLA PET NO RETORNABLE 500 ML 12,MANTENER,2024-07-01
3,MX|0030|1548015,MX|0030|112,MX|0030|522355,2024-07-11,1165,1.0,0.6000,75.9985,0030,AJEMEX,...,522355,2,F1,2025-07-01,1065,10652,112,VIDA AGUA PET NO RETORNABLE 3000 ML 6,DESARROLLAR,2024-07-01
4,MX|0030|1598515,MX|0030|84,MX|0030|517262,2024-07-17,1175,1.0,0.2365,187.1146,0030,AJEMEX,...,517262,2,F1,2025-07-01,1155,11552,84,VOLT MORA REGULAR LATA 473 ML 15,MANTENER,2024-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8640,MX|0030|1705936,MX|0030|84,MX|0030|522795,2025-06-18,1175,1.0,0.6000,94.0000,0030,AJEMEX,...,522795,2,F1,2025-07-01,1155,11552,84,VIDA AGUA PET NO RETORNABLE 1500 ML 12,OPTIMIZAR,2025-06-01
8641,MX|0030|1517017,MX|0030|84,MX|0030|517262,2025-02-26,1175,1.0,0.2365,166.7780,0030,AJEMEX,...,517262,2,F1,2025-07-01,1158,11582,84,VOLT MORA REGULAR LATA 473 ML 15,BLINDAR,2025-02-01
8642,MX|0030|1593110,MX|0030|113,MX|0030|522512,2025-05-16,1175,1.0,0.1892,148.4308,0030,AJEMEX,...,522512,2,F1,2025-07-01,1158,11582,84,VOLT PONCHE DE FRUTA LATA 473 ML 12,BLINDAR,2025-05-01
8643,MX|0030|1791962,MX|0030|112,MX|0030|599212,2025-06-26,1169,10.0,2.4000,916.0328,0030,AJEMEX,...,599212,2,F1,2025-07-01,1074,10742,112,VOLT BLUEBERRY PET NO RETORNABLE 600 ML 12,OPTIMIZAR,2025-06-01


In [69]:
pan_ventas.periodo_de_visita__c.unique()

array(['F1'], dtype=object)

In [70]:
pan_ventas.dias_de_visita__c.unique()

array(['2'], dtype=object)

In [71]:
pan_ventas.id_cliente.nunique()

214

In [72]:
pan_ventas.columns

Index(['id_cliente', 'id_sucursal', 'id_producto', 'fecha_liquidacion',
       'cod_zona', 'cant_cajafisicavta', 'cant_cajaunitvta', 'imp_netovta',
       'cod_compania', 'desc_compania', 'desc_sucursal', 'cod_pais',
       'fecha_creacion_cliente', 'cod_cliente', 'desc_marca', 'desc_formato',
       'desc_categoria', 'cod_giro', 'cod_subgiro', 'desc_giro',
       'desc_subgiro', 'fecha_proceso', 'cod_articulo_magic',
       'dias_de_visita__c', 'periodo_de_visita__c', 'ultima_visita',
       'cod_ruta', 'cod_modulo', 'cod_sucursal', 'desc_articulo',
       'new_segment', 'mes'],
      dtype='object')

In [73]:
# pan_ventas[["id_cliente", 'cod_sucursal', 'desc_sucursal','dias_de_visita__c',
#        'periodo_de_visita__c', 'ultima_visita', 'cod_ruta', 'cod_modulo']].drop_duplicates().to_csv("PS_MX_clientes_2024-10-31.csv",index=False)

In [74]:
pan_ventas.to_parquet("Processed/mexico_ventas_manana.parquet",index=False)

# 3. Pre Procesamiento

In [75]:
import pandas as pd
import numpy as np
from datetime import datetime

In [76]:
df_ventas = pd.read_parquet("Processed/mexico_ventas_manana.parquet")

In [77]:
df_ventas["fecha_liquidacion"] = pd.to_datetime(
    df_ventas["fecha_liquidacion"], format="%Y-%m-%d"
)
df_ventas["desc_marca"] = df_ventas["desc_marca"].str.strip()
df_ventas["desc_categoria"] = df_ventas["desc_categoria"].str.strip()

## 3.1 Mapeo de Pesos por Giro
Los pesos representan el orden en el que se recomendara la marca para cada cliente dependiendo del Giro

In [78]:
# Mapeo de pesos para ordenar las recomendaciones finales segun su importancia por giro
mapeo_pesos = {}
mapeo_diccionario = {}

In [79]:
df_ventas.desc_marca.unique()

array(['SPORADE', 'BIG', 'VIDA', 'VOLT', 'ROMPE', 'PULP', 'D GUSSTO',
       'CIFRUT', 'AMAYU'], dtype=object)

In [80]:
df_ventas.desc_categoria.unique()

array(['ISOTONICA', 'GASEOSAS', 'AGUA', 'ENERGIZANTE', 'NECTAR', 'CAFE',
       'JUGOS LIGEROS', 'CONSERVA DE PESCADO', 'ATUN', 'CHOCOLATE',
       'PASTAS', 'SOPA INSTANTANEA', 'PASTA'], dtype=object)

In [81]:
for giro_v in df_ventas["desc_subgiro"].unique():
    # if giro_v == "BODEGA Y PARTICULARES" or pd.isna(giro_v):
    #     mapeo_pesos = {
    #         "VOLT": 1,
    #         "CIFRUT": 2,
    #         "SPORADE": 2,
    #         "BIG": 2,
    #         "VIDA": 2,
    #         "D GUSSTO": 3,
    #         "AMAYU": 3,
    #         "PULP": 3,
    #         "ROMPE": 4,
    #         "KR": 4,
    #         "AJE AQUA": 4
    #     }
    #     mapeo_diccionario[giro_v] = mapeo_pesos
    # else:
    temp = df_ventas[(df_ventas["desc_subgiro"] == giro_v)]
    ranks = temp.groupby("desc_categoria")["cant_cajafisicavta"].sum().reset_index()
    ranks.columns = ["index","desc_categoria"]
    ranks = ranks.sort_values(by="desc_categoria", ascending=False)
    if len(list(ranks["desc_categoria"])) <= 5:
        ranks["Ranking"] = range(1, len(ranks) + 1)
    elif len(list(ranks["desc_categoria"])) > 5:
        a = list(ranks["desc_categoria"])
        b = [1, 1, 2, 2]
        # Calculamos el multiplicador para el mapeo de pesos segun la varianza
        if np.std(a) / np.mean(a) <= 1.2:
            multiplicador = 4
        else:
            multiplicador = 2
        if len(a) > 5:
            for i in range(4, len(a)):
                if a[3] <= a[i] * multiplicador:
                    b.append(3)
                else:
                    b.append(3 + i)
        ranks["Ranking"] = b

    print("*" * 20)
    print("GIRO: ", giro_v)
    print("Categorias: ", list(ranks["index"]))
    print("Counts: ", list(ranks["desc_categoria"]))
    print(ranks.set_index("index")["Ranking"].to_dict())
    mapeo_diccionario[giro_v] = ranks.set_index("index")["Ranking"].to_dict()

********************
GIRO:  GIMNASIO TRADICIONAL
Categorias:  ['ISOTONICA', 'AGUA', 'ENERGIZANTE', 'CAFE', 'JUGOS LIGEROS', 'NECTAR', 'CONSERVA DE PESCADO', 'SOPA INSTANTANEA', 'GASEOSAS']
Counts:  [652.0, 610.0, 601.0, 254.0, 50.0, 18.0, 11.0, 5.0, 3.0]
{'ISOTONICA': 1, 'AGUA': 1, 'ENERGIZANTE': 2, 'CAFE': 2, 'JUGOS LIGEROS': 7, 'NECTAR': 8, 'CONSERVA DE PESCADO': 9, 'SOPA INSTANTANEA': 10, 'GASEOSAS': 11}
********************
GIRO:  BODEGAS, TIENDAS
Categorias:  ['ENERGIZANTE', 'AGUA', 'JUGOS LIGEROS', 'CAFE', 'NECTAR', 'GASEOSAS', 'ISOTONICA', 'ATUN', 'CONSERVA DE PESCADO', 'SOPA INSTANTANEA', 'PASTA', 'CHOCOLATE']
Counts:  [6528.0, 1898.0, 1630.0, 1243.0, 1011.0, 609.0, 514.0, 88.0, 59.0, 26.0, 12.0, 8.0]
{'ENERGIZANTE': 1, 'AGUA': 1, 'JUGOS LIGEROS': 2, 'CAFE': 2, 'NECTAR': 3, 'GASEOSAS': 8, 'ISOTONICA': 9, 'ATUN': 10, 'CONSERVA DE PESCADO': 11, 'SOPA INSTANTANEA': 12, 'PASTA': 13, 'CHOCOLATE': 14}
********************
GIRO:  VENTA EN CASA PARTICULAR
Categorias:  ['ENERGIZANTE', '

## 3.2. Filtro de SKU por Ruta
Cada Ruta tiene una cantidad de SKUs distintos, la recomendacion será distinta para cada ruta, sin embargo, aquellas rutas que tengan pocos SKUs serán tratadas como una sola ruta

In [82]:
rutas = (
    df_ventas.groupby(["cod_ruta"])["id_cliente"]
    .nunique()
    .sort_values(ascending=False)
    .reset_index()["cod_ruta"]
    .unique()
)
rutas

array([1158, 1074, 1155, 1065])

In [83]:
low_sku_ruta = []

**Para rutas que tengan más de 45 SKUS**

In [84]:
for ruta in rutas:
    print("*" * 21)
    print("Ruta:", ruta)
    temp = df_ventas[(df_ventas["cod_ruta"] == ruta)]
    print("SKUs disponibles:", temp["cod_articulo_magic"].nunique())
    # print(f"Giros en ruta {ruta}:")
    # print(temp.groupby(["desc_giro"])["id_cliente"].nunique(dropna=False))
    if temp["cod_articulo_magic"].nunique() < 10:
        low_sku_ruta.append(ruta)
    else:
        temp.to_csv(f"Processed/rutas/D_{ruta}_ventas.csv", index=False)

*********************
Ruta: 1158
SKUs disponibles: 63
*********************
Ruta: 1074
SKUs disponibles: 54
*********************
Ruta: 1155
SKUs disponibles: 62
*********************
Ruta: 1065
SKUs disponibles: 52


**Para rutas que tengan menos de 45 SKUS**

In [85]:
print("*" * 21)
print("Rutas:", low_sku_ruta)
temp = df_ventas[(df_ventas["cod_ruta"].isin(low_sku_ruta))]
print("SKUs disponibles:", temp["cod_articulo_magic"].nunique())
# print(f"Giros en ruta {ruta}:")
# print(temp.groupby(["desc_giro"])["id_cliente"].nunique(dropna=False))
temp.to_parquet("Processed/rutas/D_low_ruta_ventas.parquet", index=False)

*********************
Rutas: []
SKUs disponibles: 0


# 4. Modelo Pedido Sugerido

## 4.1. Importando librerias necesarias

In [86]:
import os

from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType
from pyspark.sql.functions import (
    when,
    col,
    regexp_replace,
    concat,
    countDistinct,
    lit,
    monotonically_increasing_id,
    hash,
)

from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## 4.2 Construccion del algoritmo ALS

In [89]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 28.5 MB/s eta 0:00:00a 0:00:01


In [90]:
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix

In [91]:
def als_recommendation(cedi: str, sku_len: int):
    # === 1. Leer CSV de ventas ===
    ventas = pd.read_csv(f"Processed/rutas/D_{cedi}_ventas.csv", sep=",")
    ventas = ventas.dropna(subset=["fecha_liquidacion"])

    # === 2. Agregar frecuencia de compra ===
    ventas["id_cliente"] = ventas["cod_compania"].astype(str) + "|" + ventas["cod_cliente"].astype(str)
    ventas = (
        ventas.groupby(["id_cliente", "cod_articulo_magic"])
        .agg(frecuencia=("fecha_liquidacion", "nunique"))
        .reset_index()
    )

    # === 3. Escalar frecuencia (opcional: log o binning) ===
    # Aquí puedes aplicar reglas de negocio si quieres discretizar, por ahora mantenemos la frecuencia real como "rating"
    ventas["rating"] = ventas["frecuencia"]

    # === 4. Crear mappings numéricos ===
    user_mapping = {id_: i for i, id_ in enumerate(ventas["id_cliente"].unique())}
    item_mapping = {sku: i for i, sku in enumerate(ventas["cod_articulo_magic"].unique())}
    user_inv_map = {i: id_ for id_, i in user_mapping.items()}
    item_inv_map = {i: sku for sku, i in item_mapping.items()}

    ventas["user_id"] = ventas["id_cliente"].map(user_mapping)
    ventas["item_id"] = ventas["cod_articulo_magic"].map(item_mapping)

    # === 5. Crear matriz dispersa ===
    user_item_matrix = coo_matrix(
        (ventas["rating"], (ventas["user_id"], ventas["item_id"]))
    )

    # === 6. Entrenar modelo ALS ===
    model = AlternatingLeastSquares(
        factors=10,
        iterations=5,
        regularization=0.1,
        use_gpu=False,
        random_state=42,
    )
    
    # Convertir matriz a CSR para entrenar y recomendar
    user_item_csr = user_item_matrix.tocsr()
    item_user_csr = user_item_matrix.T.tocsr()  # 👈 también convertir la transpuesta a CSR
    
    # Entrenar
    model.fit(item_user_csr)

    # === 7. Obtener recomendaciones ===
    recommendations = []
    for user_id in range(user_item_matrix.shape[0]):
        user_items_row = item_user_csr[:, user_id].T.tocsr()  # extrae columna del user como fila
        max_items = item_user_csr.shape[0]  # total de productos únicos
        N_recs = min(sku_len, max_items)
        recs = model.recommend(user_id, user_items_row, N=N_recs)
        # for sku_id, _ in recs:
        #     recommendations.append({
        #         "id_cliente": user_inv_map[user_id],
        #         "cod_articulo_magic": item_inv_map[sku_id]
        #     })
        for rec in recs:
            sku_id = int(rec[0])  # id del producto recomendado
            # score = rec[1]  # si quieres usarlo
            recommendations.append({
                "id_cliente": user_inv_map[user_id],
                "cod_articulo_magic": item_inv_map[sku_id]
            })
    recs_df = pd.DataFrame(recommendations).drop_duplicates()

    # === 8. Guardar CSV ===
    output_path = "Output/D_rutas_rec_test.csv"
    try:
        with open(output_path, "r"):
            recs_df.to_csv(output_path, mode="a", header=False, index=False, sep=";")
    except FileNotFoundError:
        recs_df.to_csv(output_path, mode="w", header=True, index=False, sep=";")

## 4.3 Corriendo PS

In [92]:
def runALS():
    # Ruta del archivo que deseas verificar y eliminar si existe
    ruta_archivo = "Output/D_rutas_rec_test.csv"
    # Verificar si el archivo existe
    if os.path.exists(ruta_archivo):
        # Eliminar el archivo si existe
        os.remove(ruta_archivo)
        print(f"El archivo en la ruta {ruta_archivo} ha sido eliminado.")
    else:
        print(f"El archivo en la ruta {ruta_archivo} no existe.")
    # CORRIENDO ALS PARA RUTAS DE MAS DE 45 SKUs
    for ruta in rutas:
        print("*" * 21)
        temp = pd.read_csv(f"Processed/rutas/D_{ruta}_ventas.csv")
        sku_len = temp["cod_articulo_magic"].nunique()
        print(f"SKUs disponibles en ruta {ruta}:", sku_len)
        als_recommendation(ruta, sku_len)
    # CORRIENDO ALS PARA rutas que tengan menos de 45 SKUS
    if len(low_sku_ruta) != 0:
        temp = pd.read_csv(f"Processed/rutas/D_low_ruta_ventas.csv")
        sku_len = temp["cod_articulo_magic"].nunique()
        print(f"SKUs disponibles en low ruta {low_sku_ruta}:", sku_len)
        als_recommendation("low_ruta", sku_len)
    else:
        print("No existen rutas con pocos skus :D")

In [93]:
rutas = rutas[~pd.Series(rutas).isin(pd.Series(low_sku_ruta))]
rutas

array([1158, 1074, 1155, 1065])

In [94]:
%%time
runALS()

El archivo en la ruta Output/D_rutas_rec_test.csv no existe.
*********************
SKUs disponibles en ruta 1158: 63


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/5 [00:00<?, ?it/s]

IndexError: index 63 is out of bounds for axis 0 with size 63

# 5. Ajustes al output del algoritmo ALS

## 5.-9 Quedarnos con skus que hayan tenido ventas en los ultimos 30 dias

In [93]:
pan_rec = pd.read_csv("Output/D_rutas_rec_test.csv", sep=";")
pan_rec["id_cliente"] = "MX|" + pan_rec["id_cliente"]

In [96]:
pan_rec = pd.merge(pan_rec,pan_ventas[["id_cliente","cod_ruta"]].drop_duplicates(),on="id_cliente",how="left")

In [97]:
df_ventas = pd.read_parquet("Processed/mexico_ventas_manana.parquet")

In [98]:
df_ventas["fecha_liquidacion"] = pd.to_datetime(df_ventas["fecha_liquidacion"])

In [99]:
# Definir el umbral de tiempo (últimos 30 días desde hoy)
hoy = pd.Timestamp.today()
fecha_limite = hoy - pd.Timedelta(days=30)
print(fecha_limite)
# Filtrar ventas en los últimos 30 días
ventas_filtradas = df_ventas[df_ventas["fecha_liquidacion"] >= fecha_limite]

2025-06-07 17:55:08.268052


In [100]:
# Obtener los productos vendidos en cada ruta
productos_por_ruta = ventas_filtradas.groupby("cod_ruta")["cod_articulo_magic"].unique().reset_index()

In [101]:
# Unir con los productos vendidos en cada ruta
recomendaciones_validas = pan_rec.merge(productos_por_ruta, on="cod_ruta", how="inner")

In [102]:
recomendaciones_validas.columns = ['id_cliente', 'cod_articulo_magic', 'cod_ruta','lista_sku_ruta']

In [103]:
recomendaciones_validas.shape

(12478, 4)

In [104]:
recomendaciones_validas  = recomendaciones_validas[recomendaciones_validas.apply(lambda row: row["cod_articulo_magic"] in row["lista_sku_ruta"], axis=1)].reset_index(drop=True)

In [105]:
recomendaciones_validas.shape

(6524, 4)

In [106]:
recomendaciones_validas.head()

,id_cliente,cod_articulo_magic,cod_ruta,lista_sku_ruta
0,MX|0030|1597327,517140,1158,"[599212, 800060, 522353, 522795, 522352, 52263..."
1,MX|0030|1740576,517262,1158,"[599212, 800060, 522353, 522795, 522352, 52263..."
2,MX|0030|1837594,522352,1158,"[599212, 800060, 522353, 522795, 522352, 52263..."
3,MX|0030|1829807,599111,1158,"[599212, 800060, 522353, 522795, 522352, 52263..."
4,MX|0030|1604698,599111,1158,"[599212, 800060, 522353, 522795, 522352, 52263..."


In [107]:
pan_rec = recomendaciones_validas[["id_cliente","cod_articulo_magic"]].reset_index(drop=True)

In [108]:
pan_rec.head()

,id_cliente,cod_articulo_magic
0,MX|0030|1597327,517140
1,MX|0030|1740576,517262
2,MX|0030|1837594,522352
3,MX|0030|1829807,599111
4,MX|0030|1604698,599111


In [109]:
pan_rec.shape

(6524, 2)

In [110]:
pan_rec.to_parquet("Output/D_rutas_rec.parquet", index=False)

## 5.-8 Recomendar de acuerdo a Subida, Bajada, Mantener

In [111]:
pan_rec = pd.read_parquet("Output/D_rutas_rec.parquet")
#pan_rec["id_cliente"] = "MX|" + pan_rec["id_cliente"]

In [112]:
pan_rec.shape

(6524, 2)

In [113]:
pan_rec = pd.merge(pan_rec,pan_ventas[["id_cliente","cod_ruta"]].drop_duplicates(),on="id_cliente",how="left")

In [114]:
df_ventas = pd.read_parquet("Processed/mexico_ventas_manana.parquet")

In [115]:
def desplazar_nulos_fila(fila):
    return pd.Series(sorted(fila, key=pd.isna), index=fila.index)

In [116]:
# Definir una función para asignar los valores subida, mantener y bajar
def clasificar_valor(x):
    if x > 0:
        return "S"
    elif x == 0:
        return "M"
    else:
        return "B"

In [117]:
df_ventas['fecha_liquidacion'] = pd.to_datetime(df_ventas['fecha_liquidacion'])

# Definir la fecha de referencia (hoy)
fecha_actual = datetime.now(pytz.timezone("America/Lima"))#.strftime('%Y-%m-%d')
fecha_30dias_atras = (fecha_actual - pd.Timedelta(days=30)).strftime('%Y-%m-%d')
fecha_60dias_atras = (fecha_actual - pd.Timedelta(days=60)).strftime('%Y-%m-%d')

print("fecha_actual",fecha_actual)
print("fecha_30dias_atras",fecha_30dias_atras)
print("fecha_60dias_atras",fecha_60dias_atras)

fecha_actual 2025-07-07 12:55:17.263276-05:00
fecha_30dias_atras 2025-06-07
fecha_60dias_atras 2025-05-08


In [118]:
# Filtrar los últimos 30 días
df_ultimos_30 = df_ventas[(df_ventas['fecha_liquidacion'] > fecha_30dias_atras) 
                          & (df_ventas['fecha_liquidacion'] <= fecha_actual.strftime('%Y-%m-%d'))]

# Filtrar los días 31 a 60
df_31_60 = df_ventas[(df_ventas['fecha_liquidacion'] > fecha_60dias_atras)
              & (df_ventas['fecha_liquidacion'] <= fecha_30dias_atras)]

# Agrupar por codigo_usuario, ruta, codigo_producto y sumar ventas
ventas_ultimos_30 = df_ultimos_30.groupby(['cod_ruta', 'cod_articulo_magic'])['imp_netovta'].sum().reset_index()
ventas_ultimos_30["mes"] = "0_30"
ventas_31_60 = df_31_60.groupby(['cod_ruta', 'cod_articulo_magic'])['imp_netovta'].sum().reset_index()
ventas_31_60["mes"] = "31_60"

df_grouped = pd.concat([ventas_ultimos_30,ventas_31_60],axis=0,ignore_index=True)
df_grouped = pd.pivot_table(
    df_grouped,
    values="imp_netovta",
    index=["cod_ruta", "cod_articulo_magic"],
    columns=["mes"],
    aggfunc="sum",
).reset_index()

# Rellenando con 0 si es nulo
df_grouped["0_30"] = df_grouped["0_30"].fillna(0)
df_grouped["31_60"] = df_grouped["31_60"].fillna(0)

# Asignando variacion de productos por mes
df_grouped["v1_2"] = (
    (df_grouped[df_grouped.columns[2]] - df_grouped[df_grouped.columns[3]])
    / df_grouped[df_grouped.columns[3]]
    * 100
)
df_grouped["v1_2"].replace([float('inf'), float('-inf')], -1, inplace=True)
df_grouped["v1_2"] = df_grouped["v1_2"].fillna(-1)
df_grouped["vp"] = (df_grouped["v1_2"])

/tmp/ipykernel_14092/261865676.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_grouped["v1_2"].replace([float('inf'), float('-inf')], -1, inplace=True)


In [119]:
# Aplicar la función a la columna 'valor' y guardar el resultado en una nueva columna 'flag'
df_grouped["flag"] = df_grouped["vp"].apply(lambda x: clasificar_valor(x))
# Definir el mapeo de valores
mapeo_flag = {"S": 0, "M": 1, "B": 2}
# Aplicar el mapeo a la columna 'clasificacion' y guardar el resultado en una nueva columna 'flag_rank'
df_grouped["flag_rank"] = df_grouped["flag"].map(mapeo_flag)

In [120]:
df_grouped

mes,cod_ruta,cod_articulo_magic,0_30,31_60,v1_2,vp,flag,flag_rank
0,1065,500264,478.6964,0.0000,-1.000000,-1.000000,B,2
1,1065,500354,342.5946,0.0000,-1.000000,-1.000000,B,2
2,1065,509826,930.0500,247.5000,275.777778,275.777778,S,0
3,1065,516209,67.1109,119.5617,-43.869232,-43.869232,B,2
4,1065,517140,770.9976,582.9140,32.266098,32.266098,S,0
...,...,...,...,...,...,...,...,...
125,1158,599590,857.3644,0.0000,-1.000000,-1.000000,B,2
126,1158,800054,400.0000,0.0000,-1.000000,-1.000000,B,2
127,1158,800058,540.0000,0.0000,-1.000000,-1.000000,B,2
128,1158,800060,225.0000,75.0000,200.000000,200.000000,S,0


In [121]:
pan_rec.shape

(6524, 3)

In [122]:
pan_rec = pd.merge(pan_rec,df_grouped[["cod_ruta", "cod_articulo_magic","flag_rank"]],on=["cod_ruta", "cod_articulo_magic"],how="left")

In [123]:
pan_rec["flag_rank"] = pan_rec["flag_rank"].fillna(3)

In [124]:
# Agregar una columna con el orden original
pan_rec["original_order"] = pan_rec.index

In [125]:
# Ordenar por cod_cliente, rank_producto (ascendente) y mantener el orden original en caso de empate
pan_rec = pan_rec.sort_values(by=["id_cliente", "flag_rank", "original_order"], ascending=[True, True, True])#.drop(columns=["original_order"])

In [126]:
pan_rec.head()

,id_cliente,cod_articulo_magic,cod_ruta,flag_rank,original_order
3597,MX|0030|1510228,522632,1155,0,3597
3668,MX|0030|1510228,599587,1155,0,3668
3705,MX|0030|1510228,598901,1155,0,3705
3809,MX|0030|1510228,522713,1155,0,3809
3843,MX|0030|1510228,522512,1155,0,3843


In [127]:
pan_rec.shape

(6524, 5)

In [128]:
pan_rec = pan_rec[["id_cliente","cod_articulo_magic"]].reset_index(drop=True)

In [129]:
pan_rec.head()

,id_cliente,cod_articulo_magic
0,MX|0030|1510228,522632
1,MX|0030|1510228,599587
2,MX|0030|1510228,598901
3,MX|0030|1510228,522713
4,MX|0030|1510228,522512


In [130]:
pan_rec.shape

(6524, 2)

In [131]:
pan_rec.to_parquet("Output/D_rutas_rec.parquet", index=False)

## 5.-7 Usamos el archivo de validacion para no recomendar SKUs que no se deben a las rutas

In [132]:
pan_rec = pd.read_parquet("Output/D_rutas_rec.parquet")
# pan_rec["id_cliente"] = "MX|" + pan_rec["id_cliente"]

In [133]:
pan_rec.shape

(6524, 2)

In [134]:
# Ruta del archivo en S3
s3_path = "s3://aje-prd-analytics-artifacts-s3/pedido_sugerido/data-v1/mexico/maestro_productos_mexico000"

# Leer el CSV directamente a un DataFrame
skus_val = pd.read_csv(s3_path,sep = ";")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:294: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [135]:
skus_val = skus_val[skus_val.cod_compania==30]
# Nos quedamos solo con las filas del proceso de hoy
# Obtener la fecha de hoy en el mismo formato
hoy = int(datetime.now(pytz.timezone("America/Lima")).strftime('%Y%m%d'))
print("fecha_proceso: ", hoy)
# Filtrar solo las filas con la fecha de hoy
# skus_val = skus_val[skus_val['fecha_proceso'] == hoy]
skus_val["cod_compania"] = (skus_val["cod_compania"].astype(str).apply(lambda x: str(int(x)).rjust(4, "0")))
skus_val["id_cliente"] = (
    "MX"
    + "|"
    + skus_val["cod_compania"].astype(str)
    + "|"
    + skus_val["cod_cliente"].astype(str)
)

fecha_proceso:  20250707


In [136]:
skus_val.shape

(919953, 14)

In [137]:
pan_rec.groupby("id_cliente").cod_articulo_magic.nunique().mean()

30.485981308411215

In [138]:
pan_rec = pd.merge(pan_rec,skus_val[['cod_articulo_magic', 'id_cliente']].drop_duplicates(),
                   left_on=["id_cliente","cod_articulo_magic"],right_on=["id_cliente","cod_articulo_magic"],how="inner")

In [139]:
pan_rec.shape

(1578, 2)

In [140]:
pan_rec.groupby("id_cliente").cod_articulo_magic.nunique().mean()

7.373831775700935

In [141]:
pan_rec = pan_rec[["id_cliente","cod_articulo_magic"]]

In [142]:
pan_rec.head()

,id_cliente,cod_articulo_magic
0,MX|0030|1510228,522632
1,MX|0030|1510228,599587
2,MX|0030|1510228,598901
3,MX|0030|1510228,522713
4,MX|0030|1510228,522512


In [143]:
pan_rec.shape

(1578, 2)

In [144]:
pan_rec.to_parquet("Output/D_rutas_rec.parquet", index=False)

## 5.-5 Cruzar recomendaciones con df_quiebres (STOCK)

In [145]:
# pan_rec = pd.read_csv("Output/D_rutas_rec.csv", sep=";")
# pan_rec["id_cliente"] = "MX|" + pan_rec["id_cliente"]

In [146]:
# df_quiebres.cod_articulo_magic.nunique()

In [147]:
# pan_rec.cod_articulo_magic.nunique()

In [148]:
# pan_rec[
#     pan_rec.cod_articulo_magic.isin(df_quiebres.cod_articulo_magic.unique())
# ].cod_articulo_magic.nunique()

In [149]:
# pan_ventas = pd.read_csv("Processed/ventas_mexico_12m.csv")

In [150]:
# df_quiebres["cod_sucursal"] = df_quiebres["id_sucursal"].str.split("|").str[1]
# df_quiebres["cod_articulo_magic"] = df_quiebres["id_articulo"].str.split("|").str[1]

In [151]:
# df_quiebres.cant_cajaunitaria = df_quiebres.cant_cajaunitaria.astype(float)
# df_quiebres.cod_sucursal = df_quiebres.cod_sucursal.astype(int)
# df_quiebres.cod_articulo_magic = df_quiebres.cod_articulo_magic.astype(int)

In [152]:
# pan_ventas = pan_ventas.sort_values(["fecha_liquidacion"])

In [153]:
# fecha_compra_sku = (
#     pan_ventas[["fecha_liquidacion", "cod_sucursal", "cod_articulo_magic"]]
#     .drop_duplicates()
#     .reset_index(drop=True)
# )

In [154]:
# # Calculando ultimas 3 fechas de comrpas por sku por sucurusal
# count_dias_atras = 3
# ultimas_3_fechas_sku = (
#     fecha_compra_sku.groupby(["cod_sucursal", "cod_articulo_magic"])
#     .tail(count_dias_atras)
#     .sort_values(["cod_articulo_magic", "cod_sucursal", "fecha_liquidacion"])
#     .reset_index(drop=True)
# )

In [155]:
# # Se combinan df de ventas y df de 3 ultimas fechas de sku por sucursal
# ventas_3_ultimas_fechas = pd.merge(
#     pan_ventas,
#     ultimas_3_fechas_sku,
#     on=["fecha_liquidacion", "cod_sucursal", "cod_articulo_magic"],
#     how="inner",
# )
# # Se extraen las cajas unitarias compradas por cada fecha
# # ventas_3_ultimas_fechas=ventas_3_ultimas_fechas.groupby(['cod_sucursal', 'fecha_liquidacion'])["cod_articulo_magic"].value_counts().reset_index().sort_values(["cod_articulo_magic","cod_sucursal","fecha_liquidacion"])
# ventas_3_ultimas_fechas = (
#     ventas_3_ultimas_fechas.groupby(
#         ["cod_sucursal", "fecha_liquidacion", "cod_articulo_magic"]
#     )["cant_cajaunitvta"]
#     .sum()
#     .reset_index()
#     .sort_values(["cod_articulo_magic", "cod_sucursal", "fecha_liquidacion"])
# )
# # Se calcula el promedio de las 3 ultimas fechas por sku y sucursal
# ventas_3_ultimas_fechas = (
#     ventas_3_ultimas_fechas.groupby(["cod_sucursal", "cod_articulo_magic"])[
#         "cant_cajaunitvta"
#     ]
#     .mean()
#     .reset_index()
#     .sort_values(["cod_articulo_magic", "cod_sucursal"])
# )
# ventas_3_ultimas_fechas.columns = [
#     "cod_sucursal",
#     "cod_articulo_magic",
#     "promedio_caja_unit",
# ]
# ventas_3_ultimas_fechas = ventas_3_ultimas_fechas.sort_values(
#     ["cod_articulo_magic", "cod_sucursal"]
# ).reset_index(drop=True)
# ventas_3_ultimas_fechas

In [156]:
# # Obtener la fecha de hoy
# fecha_hoy = datetime.now(pytz.timezone("America/Lima")).strftime("%Y-%m-%d")
# # fecha_hoy = "2024-08-25"
# fecha_hoy

In [157]:
# # Combinar ambos DF
# result_df = pd.merge(
#     df_quiebres[(df_quiebres["fecha_inventario"] == fecha_hoy)],
#     ventas_3_ultimas_fechas,
#     on=["cod_sucursal", "cod_articulo_magic"],
#     how="left",
# )
# result_df["ps_dias"] = result_df["cant_cajaunitaria"] / result_df["promedio_caja_unit"]
# result_df.loc[result_df["cant_cajaunitaria"] == 0, "ps_dias"] = 0
# result_df.loc[result_df["ps_dias"].isnull(), "ps_dias"] = 1
# # result_df.loc[(result_df['ps_dias'].isnull())&(result_df['cant_cajaunitaria'] >= 100), 'ps_dias'] = 2.22

In [158]:
# df_stock = result_df[result_df["ps_dias"] >= 1]
# df_stock

In [159]:
# df_stock = pd.concat(
#     [
#         df_stock,
#         result_df[
#             (result_df.cant_cajaunitaria == 0)
#             & (~result_df.promedio_caja_unit.isnull())
#         ].reset_index(drop=True),
#     ],
#     axis=0,
# )
# df_stock

In [160]:
# pan_rec = pd.merge(
#     pan_rec,
#     pan_ventas[["id_cliente", "cod_sucursal"]].drop_duplicates(),
#     on="id_cliente",
#     how="left",
# )

In [161]:
# pan_rec

In [162]:
# final_pan_rec = pd.DataFrame()
# for sucursal in df_stock.cod_sucursal.unique():
#     temp = pan_rec[pan_rec["cod_sucursal"] == sucursal]
#     # Lista de skus con stock para 2 dias
#     list_sku_2_days = df_stock[(df_stock["cod_sucursal"] == sucursal)][
#         "cod_articulo_magic"
#     ].unique()
#     temp = temp[temp["cod_articulo_magic"].isin(list_sku_2_days)]
#     final_pan_rec = pd.concat([final_pan_rec, temp], axis=0)
#     final_pan_rec = final_pan_rec.reset_index(drop=True)
# final_pan_rec

In [163]:
# final_pan_rec.groupby("cod_sucursal").agg(
#     {"cod_articulo_magic": "nunique", "id_cliente": "nunique"}
# )

In [164]:
# final_pan_rec.id_cliente.nunique()

In [165]:
# final_pan_rec.cod_articulo_magic.nunique()

In [166]:
# final_pan_rec.to_csv("Output/D_rutas_rec.csv", sep=";", index=False)

## 5.-4 Mantener Solo SKUS en especifico (solo usar si se tiene el excel con SKU activos -> el input de ventas ya deberia incluir este filtro)

In [167]:
pan_rec=pd.read_parquet("Output/D_rutas_rec.parquet")

In [168]:
pan_rec.groupby("id_cliente")["cod_articulo_magic"].nunique().mean()

7.373831775700935

In [169]:
pan_rec.groupby("id_cliente")["cod_articulo_magic"].count().mean()

7.373831775700935

In [170]:
sku_con_precio=pd.read_excel("Input/MX_SKUS.xlsx",sheet_name="Hoja2")

In [171]:
# Original size de recomendacioness
pan_rec.shape

(1578, 2)

In [172]:
pan_rec = pan_rec[pan_rec.cod_articulo_magic.isin(sku_con_precio["COD_SKUS"].unique())].reset_index(drop=True)

In [173]:
# Size de recomendaciones luego de filtar solo a los del excel de SKUs activos
pan_rec.shape

(1436, 2)

In [174]:
pan_rec.groupby("id_cliente")["cod_articulo_magic"].count().mean()

6.710280373831775

In [175]:
pan_rec["cod_articulo_magic"].nunique()

38

In [176]:
pan_rec.to_parquet("Output/D_rutas_rec.parquet", index=False)

## 5.-3 Quitar Recomendaciones de SKUS en especifico

In [177]:
# import os
# import re
# import pandas as pd

In [178]:
# pan_rec=pd.read_csv("Output/D_rutas_rec.csv", sep=";")

In [179]:
# skus_sin_precio=[622398]

In [180]:
# pan_rec=pan_rec[~(pan_rec["cod_articulo_magic"].isin(skus_sin_precio))].reset_index(drop=True)

In [181]:
# pan_rec.to_csv("Output/D_rutas_rec.csv", sep=";", index=False)

## 5.-2 Quitar Recomendaciones de los ultimos 14 dias

In [182]:
pan_rec=pd.read_parquet("Output/D_rutas_rec.parquet")

In [183]:
master_prod=pd.read_csv("Input/MX_maestro_productos.csv")
master_prod

,cod_articulo_magic,desc_articulo
0,25397,JARABE TERMINADO BIG COUNTRY
1,25941,BASE DE BEBIDA GASIFICADA LIGHT TORONJA BL-600...
2,26418,BOTELLA SOPLADA-ETIQUETADA DE BIG COLA 620 ML ...
3,26419,BOTELLA SOPLADA-ETIQUETADA DE BIG COLA 620 C/P...
4,31384,BASE DE BEBIDA FRUIT PUNCH BF-MX-90088/1U (GUA...
...,...,...
8244,81248,LONA TERMINADA VOLT FAMILY SUBE LA VIBE 0.37 X...
8245,81246,ESQUINERO PALLET MULTIMARCA 0.80 X 0.80
8246,623924,D GUSSTO COPOS SABOR CHOCOLATE BOLSA 150 GRS 1
8247,623918,D GUSSTO COPOS SABOR CHOCOLATE BOLSA 30 GRS 48


In [184]:
fecha_tomorrow = (
    datetime.now(pytz.timezone("America/Lima")) + timedelta(days=1)
).strftime("%Y-%m-%d")
fecha_tomorrow

'2025-07-08'

In [185]:
ruta_archivo = f"Output/PS_piloto_v1/D_base_pedidos_{fecha_tomorrow}.csv"
ruta_archivo2 = f"Output/PS_piloto_v1/D_base_pedidos_{fecha_tomorrow}.parquet"

# Verificar si el archivo existe
if os.path.exists(ruta_archivo):
    # Eliminar el archivo si existe
    os.remove(ruta_archivo)
    print(f"El archivo en la ruta {ruta_archivo} ha sido eliminado.")
else:
    print(f"El archivo en la ruta {ruta_archivo} no existe.")
    
# Verificar si el archivo existe
if os.path.exists(ruta_archivo2):
    # Eliminar el archivo si existe
    os.remove(ruta_archivo2)
    print(f"El archivo en la ruta {ruta_archivo2} ha sido eliminado.")
else:
    print(f"El archivo en la ruta {ruta_archivo2} no existe.")

El archivo en la ruta Output/PS_piloto_v1/D_base_pedidos_2025-07-08.csv no existe.
El archivo en la ruta Output/PS_piloto_v1/D_base_pedidos_2025-07-08.parquet no existe.


In [186]:
# Directorio donde se encuentran los archivos
directorio = 'Output/PS_piloto_v1/'

# Expresión regular para encontrar fechas en el nombre de los archivos
patron_fecha = r'\d{4}-\d{2}-\d{2}'

# Lista para almacenar las fechas extraídas
fechas = []

# Iterar sobre los archivos en el directorio
for archivo in os.listdir(directorio):
    # Verificar si el archivo es un archivo CSV y coincide con el patrón de nombres
    if archivo.endswith('.csv') and re.match(r'^D_base_pedidos_\d{4}-\d{2}-\d{2}\.csv$', archivo):
        # Extraer la fecha del nombre del archivo
        fecha = re.search(patron_fecha, archivo).group()
        # Agregar la fecha a la lista
        fechas.append(fecha)

In [187]:
last_7_days_2=sorted(fechas)[-14:]
#last_7_days_2=last_7_days_2[:-1]
last_7_days_2

['2025-06-07',
 '2025-06-09',
 '2025-06-10',
 '2025-06-11',
 '2025-06-12',
 '2025-06-13',
 '2025-06-14',
 '2025-06-16',
 '2025-06-17',
 '2025-06-18',
 '2025-06-19',
 '2025-06-20',
 '2025-06-21',
 '2025-06-23']

In [188]:
#Leer recomendaciones de los ultimos 7 dias y cruzar con clientes que tienen recomendacion para mañana
last_7_days_recs=pd.DataFrame()
for fecha_rec in last_7_days_2:
    df_temp=pd.read_csv(f"Output/PS_piloto_v1/D_base_pedidos_{fecha_rec}.csv",dtype={"Compania":"str","Cliente":"str"})
    df_temp["id_cliente"]='MX|'+df_temp['Compania']+'|'+df_temp['Cliente']
    df_temp=df_temp[df_temp["id_cliente"].isin(pan_rec["id_cliente"].unique())]
    last_7_days_recs=pd.concat([last_7_days_recs,df_temp],axis=0)
    print(f"{fecha_rec} done")

2025-06-07 done
2025-06-09 done
2025-06-10 done
2025-06-11 done
2025-06-12 done
2025-06-13 done
2025-06-14 done
2025-06-16 done
2025-06-17 done
2025-06-18 done
2025-06-19 done
2025-06-20 done
2025-06-21 done
2025-06-23 done


In [189]:
# Utilizamos merge para combinar los dataframes, utilizando la columna 'Cliente' como clave
df_combinado = pd.merge(pan_rec, last_7_days_recs, left_on=['id_cliente', 'cod_articulo_magic'],right_on=['id_cliente', 'Producto'], how='left', indicator=True)
# Filtramos los registros que solo están en el DataFrame 1
df_resultado = df_combinado[df_combinado['_merge'] == 'left_only'][["id_cliente","cod_articulo_magic"]].reset_index(drop=True)
df_resultado

,id_cliente,cod_articulo_magic
0,MX|0030|1510228,522632
1,MX|0030|1510228,599587
2,MX|0030|1510228,598901
3,MX|0030|1510228,522512
4,MX|0030|1510228,517262
...,...,...
943,MX|0030|1837592,522352
944,MX|0030|1837593,522353
945,MX|0030|1837594,522352
946,MX|0030|1837595,522352


In [190]:
pan_rec.shape

(1436, 2)

In [191]:
df_resultado.to_parquet("Output/D_rutas_rec.parquet",index=False)

## 5.0 Calcular Irregularidad de clientes

In [192]:
pan_rec = pd.read_parquet("Output/D_rutas_rec.parquet")

In [193]:
pan_rec["id_cliente"].nunique()

194

In [194]:
now = pd.to_datetime(datetime.now(pytz.timezone("America/Lima")).strftime("%Y-%m-01"))
fecha_doce_meses_atras = now - pd.DateOffset(months=12)
lista_m12 = [fecha_doce_meses_atras + pd.DateOffset(months=i) for i in range(12)]
lista_m12

fecha_seis_meses_atras = now - pd.DateOffset(months=6)
lista_m6 = [fecha_seis_meses_atras + pd.DateOffset(months=i) for i in range(6)]
lista_m6

[Timestamp('2025-01-01 00:00:00'),
 Timestamp('2025-02-01 00:00:00'),
 Timestamp('2025-03-01 00:00:00'),
 Timestamp('2025-04-01 00:00:00'),
 Timestamp('2025-05-01 00:00:00'),
 Timestamp('2025-06-01 00:00:00')]

In [195]:
temp_df_ventas = df_ventas[df_ventas["id_cliente"].isin(pan_rec["id_cliente"].unique())]

In [196]:
qw = (
    temp_df_ventas[["id_cliente", "mes"]]
    .drop_duplicates()
    .sort_values(["id_cliente", "mes"])
    .groupby("id_cliente")
    .tail(12)
    .reset_index(drop=True)
)
qw["mes"] = pd.to_datetime(qw["mes"])
qw["m12"] = qw["mes"].isin(lista_m12)
qw["m6"] = qw["mes"].isin(lista_m6)

In [197]:
categoria_cliente = qw.groupby("id_cliente")[["m12", "m6"]].sum().reset_index()

In [198]:
# Definir condiciones
condicion_super = categoria_cliente["m12"] == 12
condicion_frecuente = (
    (categoria_cliente["m12"] < 12)
    & (categoria_cliente["m12"] >= 6)
    & (categoria_cliente["m6"] == 6)
)
condicion_regulares = (categoria_cliente["m6"] <= 6) & (categoria_cliente["m12"] <= 6)
condicion_riesgo = (categoria_cliente["m6"] < 6) & (categoria_cliente["m6"] >= 4)
condicion_irregulares = categoria_cliente["m6"] < 4

# Aplicar condiciones y asignar categorías
categoria_cliente.loc[condicion_super, "categoria_cliente"] = "Super"
categoria_cliente.loc[condicion_frecuente, "categoria_cliente"] = "Frecuente"
categoria_cliente.loc[condicion_regulares, "categoria_cliente"] = "Regular"
categoria_cliente.loc[condicion_riesgo, "categoria_cliente"] = "Riesgo"
categoria_cliente.loc[condicion_irregulares, "categoria_cliente"] = "Irregular"

**Recalcular Irregularidad**

In [199]:
# Definir condiciones
condicion_regular = (
    (categoria_cliente["categoria_cliente"] == "Super")
    | (categoria_cliente["categoria_cliente"] == "Frecuente")
    | (categoria_cliente["categoria_cliente"] == "Regular")
    | (categoria_cliente["categoria_cliente"] == "Riesgo")
)
condicion_irregular = categoria_cliente["categoria_cliente"] == "Irregular"
# Aplicar condiciones y asignar categorías
categoria_cliente.loc[condicion_regular, "categoria_cliente_2"] = "Regular"
categoria_cliente.loc[condicion_irregular, "categoria_cliente_2"] = "Irregular"

In [200]:
categoria_cliente

,id_cliente,m12,m6,categoria_cliente,categoria_cliente_2
0,MX|0030|1510228,12,6,Super,Regular
1,MX|0030|1517017,12,6,Super,Regular
2,MX|0030|1524033,10,4,Riesgo,Regular
3,MX|0030|1545235,9,3,Irregular,Irregular
4,MX|0030|1545236,10,4,Riesgo,Regular
...,...,...,...,...,...
189,MX|0030|1837592,1,1,Irregular,Irregular
190,MX|0030|1837593,2,2,Irregular,Irregular
191,MX|0030|1837594,3,3,Irregular,Irregular
192,MX|0030|1837595,2,2,Irregular,Irregular


In [201]:
categoria_cliente["categoria_cliente"].value_counts(dropna=False)

categoria_cliente
Irregular    68
Riesgo       59
Super        36
Frecuente    31
Name: count, dtype: int64

In [202]:
categoria_cliente["categoria_cliente_2"].value_counts(dropna=False)

categoria_cliente_2
Regular      126
Irregular     68
Name: count, dtype: int64

## 5.1. Obteniendo marcas de recomendaciones por cliente

In [203]:
pan_rec = pd.read_parquet("Output/D_rutas_rec.parquet")

In [204]:
pan_ventas = df_ventas[(df_ventas["cod_ruta"].isin(rutas))]
pan_ventas2 = df_ventas[(df_ventas["cod_ruta"].isin(low_sku_ruta))]
pan_ventas = pd.concat([pan_ventas, pan_ventas2], axis=0)

In [205]:
marca_articulo = pan_ventas[["desc_categoria", "cod_articulo_magic"]].drop_duplicates()
print(marca_articulo.shape)
marca_articulo.head()

(69, 2)


,desc_categoria,cod_articulo_magic
0,ISOTONICA,516209
1,GASEOSAS,500264
3,AGUA,522355
4,ENERGIZANTE,517262
6,ENERGIZANTE,517263


In [206]:
cliente_rec_marca = pd.merge(
    pan_rec, marca_articulo, on="cod_articulo_magic", how="left"
)
cliente_rec_marca["desc_categoria"] = cliente_rec_marca["desc_categoria"].str.strip()
print(cliente_rec_marca.shape)
cliente_rec_marca

(948, 3)


,id_cliente,cod_articulo_magic,desc_categoria
0,MX|0030|1510228,522632,NECTAR
1,MX|0030|1510228,599587,ISOTONICA
2,MX|0030|1510228,598901,JUGOS LIGEROS
3,MX|0030|1510228,522512,ENERGIZANTE
4,MX|0030|1510228,517262,ENERGIZANTE
...,...,...,...
943,MX|0030|1837592,522352,AGUA
944,MX|0030|1837593,522353,AGUA
945,MX|0030|1837594,522352,AGUA
946,MX|0030|1837595,522352,AGUA


## 5.2. Calcular marcas distintas recomendadas por cliente

In [207]:
# Conteo de categorias recomendadas por Cliente
cods2 = cliente_rec_marca.groupby("id_cliente")["desc_categoria"].nunique().reset_index()
print(cods2.shape)
cods2.head()

(194, 2)


,id_cliente,desc_categoria
0,MX|0030|1510228,5
1,MX|0030|1517017,5
2,MX|0030|1524033,2
3,MX|0030|1545235,4
4,MX|0030|1545236,2


## 5.3 Quitando los SKUs de las ultimas 2 semanas (Evitar recompra)

In [208]:
last_2_weeks = pd.to_datetime(datetime.now(pytz.timezone("America/Lima"))) - pd.DateOffset(
    days=14
)
last_2_weeks

Timestamp('2025-06-23 12:55:40.412963-0500', tz='America/Lima')

In [209]:
# Asegurarse de que 'fecha_liquidacion' esté en formato datetime
pan_ventas["fecha_liquidacion"] = pd.to_datetime(pan_ventas["fecha_liquidacion"])
# Convertir 'fecha_liquidacion' a la misma zona horaria que 'last_year'
pan_ventas["fecha_liquidacion"] = pan_ventas["fecha_liquidacion"].dt.tz_localize(
    "America/Lima", nonexistent="NaT", ambiguous="NaT"
)

In [210]:
df_quitar_2_weeks = pan_ventas[pan_ventas["fecha_liquidacion"]>=last_2_weeks][["id_cliente","cod_articulo_magic"]].drop_duplicates().reset_index(drop=True)

In [211]:
# Hacemos un merge para identificar coincidencias
cliente_rec_sin4 = cliente_rec_marca.merge(df_quitar_2_weeks, on=['id_cliente', 'cod_articulo_magic'], how='left', indicator=True)
# Filtramos solo los registros que NO están en df_quitar_2_weeks
cliente_rec_sin4 = cliente_rec_sin4[cliente_rec_sin4['_merge'] == 'left_only'].drop(columns=['_merge'])

In [212]:
cliente_rec_marca.id_cliente.nunique()

194

In [213]:
cliente_rec_sin4.id_cliente.nunique()

185

In [214]:
cliente_rec_sin4

,id_cliente,cod_articulo_magic,desc_categoria
0,MX|0030|1510228,522632,NECTAR
1,MX|0030|1510228,599587,ISOTONICA
3,MX|0030|1510228,522512,ENERGIZANTE
4,MX|0030|1510228,517262,ENERGIZANTE
5,MX|0030|1510228,598801,ISOTONICA
...,...,...,...
943,MX|0030|1837592,522352,AGUA
944,MX|0030|1837593,522353,AGUA
945,MX|0030|1837594,522352,AGUA
946,MX|0030|1837595,522352,AGUA


## 5.4 Filtro para calcula antiguedad de clientes

In [215]:
pan_ventas = pan_ventas.rename(columns={"fecha_creacion_cliente": "fecha_creacion"})

In [216]:
# Convertir la columna 'fecha_creacion' a tipo fecha
pan_ventas["fecha_creacion"] = pd.to_datetime(
    pan_ventas["fecha_creacion"], format="%Y%m%d"
)

In [217]:
# Definir una función para etiquetar los clientes según su fecha de creación
def etiquetar_cliente(fecha_creacion):
    if pd.isnull(fecha_creacion):
        return (
            "nf"  # Si la fecha de creación es nula, etiquetar como 'nf' (no encontrada)
        )
    else:
        hoy = datetime.now()  # Obtener la fecha actual
        hace_12_meses = hoy - timedelta(days=365)  # Calcular hace 12 meses
        if fecha_creacion >= hace_12_meses:
            return "new_client"  # Si la fecha de creación está dentro de los últimos 12 meses, etiquetar como 'new_client'
        else:
            return "old_client"  # Si la fecha de creación es anterior a los últimos 12 meses, etiquetar como 'old_client'

In [218]:
# Aplicar la función a la columna 'fecha_creacion' para crear la nueva columna 'tipo_cliente'
pan_ventas["antiguedad"] = pan_ventas["fecha_creacion"].apply(etiquetar_cliente)

In [219]:
pan_antiguedad_clientes = (
    pan_ventas[["id_cliente", "fecha_creacion", "antiguedad"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

## 5.4.2 Obteniendo datos necesarios para armar DF para subir a SalesForce

In [220]:
# pan_ventas.groupby(["id_cliente","cod_compania","cod_sucursal","cod_cliente","cod_modulo"])[["cant_cajafisicavta","cant_cajaunitvta"]].sum().reset_index()

In [221]:
datos_para_salesforce = (
    pan_ventas.groupby(
        ["id_cliente", "cod_compania", "cod_sucursal", "cod_cliente", "cod_ruta"]
    )[
        [
            "id_cliente",
            "cod_compania",
            "cod_sucursal",
            "cod_cliente",
            "cod_modulo",
            "cod_ruta",
        ]
    ]
    .head(1)
    .reset_index(drop=True)
)

## 5.5 Juntando las Recomendaciones Finales
 - Ordenado de acuerdo al peso de la recomendación y el id de usuario.
 - Hay 5 skus de 5 marcas distintas que se recomiendan a cada usuario
 - Ningún SKU que haya comprado el usuario en sus ultimas X visitas será recomendado. La cantidad de visitas (X) a ignorar depende de sus visitas promedio mensuales.

In [222]:
# Obtener primer SKU de cada marca de la recomendacion
final_rec = cliente_rec_sin4.groupby(["id_cliente", "desc_categoria"]).first().reset_index()
# Agregar Marca de sus primeras compras
# final_rec=pd.merge(final_rec,pan_evo_venta[["id_cliente","mes_1_marcaCount","mes_1_marca","mes_2_marca","mes_3_marca","mes_4_marca","mes_5_marca","mes_6_marca","len_m","tipo_cliente"]],how='left',on='id_cliente')
final_rec = pd.merge(final_rec, cods2, how="left", on="id_cliente")
# Agregar descripcion de SKU recomendado
pan_prod = pan_prod[["cod_articulo_magic", "desc_articulo"]].drop_duplicates()
pan_prod = pan_prod.groupby(["cod_articulo_magic"]).first().reset_index()
final_rec = pd.merge(final_rec, pan_prod, how="left", on="cod_articulo_magic")
# Cambiar nombre de columnas
# final_rec.columns=['id_cliente','marca_rec','sku','len_marca_origen','mes_1_marca',"mes_2_marca","mes_3_marca","mes_4_marca","mes_5_marca","mes_6_marca",'len_mes_compras','tipo','len_marca_rec','desc_rec']
final_rec.columns = ["id_cliente", "marca_rec", "sku", "len_marca_rec", "desc_rec"]
# Agregar Giro del cliente
giros = df_ventas[df_ventas["id_cliente"].isin(final_rec["id_cliente"].unique())][
    ["id_cliente", "desc_giro", "desc_subgiro"]
].drop_duplicates()
final_rec = pd.merge(final_rec, giros, on="id_cliente", how="left")
# Agregar si el cliente es Regular o Irregular
final_rec = pd.merge(
    final_rec,
    categoria_cliente[["id_cliente", "categoria_cliente_2"]],
    on="id_cliente",
    how="left",
)
final_rec.loc[final_rec["categoria_cliente_2"] == "Irregular", "desc_giro"] = np.nan
# Agregar antiguedad de cliente
final_rec = pd.merge(final_rec, pan_antiguedad_clientes, how="left", on="id_cliente")
# Agregar segmento de cliente
info_segmentos = pan_ventas[
    [
        "id_cliente",
        "new_segment",
        "dias_de_visita__c",
        "periodo_de_visita__c",
        "ultima_visita",
    ]
].drop_duplicates()
info_segmentos = info_segmentos.groupby(["id_cliente"]).first().reset_index()
final_rec = pd.merge(final_rec, info_segmentos, how="left", on="id_cliente")
# Agregar datos necesarios para SALESFORCE
final_rec = pd.merge(final_rec, datos_para_salesforce, how="left", on="id_cliente")
# Agregar Peso de marcas dependiendo del giro del cliente
# final_rec["peso"]= final_rec['marca_rec'].replace(mapeo_pesos)
# final_rec['peso'] = final_rec.apply(lambda row: mapeo_diccionario[row['desc_giro']].get(row['marca_rec'], 5), axis=1)
final_rec["peso"] = final_rec.apply(
    lambda row: mapeo_diccionario.get(row["desc_subgiro"], {}).get(row["marca_rec"], 5),
    axis=1,
)

In [223]:
# Ordenar la recomendacion de acuerdo al peso de la marca y nos quedamos con las 5 primeras marcas
final_rec = (
    final_rec.sort_values(["id_cliente", "peso"]).groupby(["id_cliente"]).head(5)
)
# Agregamos rank de la marca recomendada
final_rec["marca_rec_rank"] = final_rec.groupby("id_cliente").cumcount() + 1
final_rec = final_rec.reset_index(drop=True)

In [224]:
final_rec["sku"] = final_rec["sku"].astype(int)
final_rec["peso"] = final_rec["peso"].astype(int)
final_rec["marca_rec_rank"] = final_rec["marca_rec_rank"].astype(int)

In [225]:
final_rec

,id_cliente,marca_rec,sku,len_marca_rec,desc_rec,desc_giro,desc_subgiro,categoria_cliente_2,fecha_creacion,antiguedad,...,dias_de_visita__c,periodo_de_visita__c,ultima_visita,cod_compania,cod_sucursal,cod_cliente,cod_modulo,cod_ruta,peso,marca_rec_rank
0,MX|0030|1510228,AGUA,522355,5,VIDA AGUA PET NO RETORNABLE 3000 ML 6,BODEGA Y PARTICULARES,"BODEGAS, TIENDAS",Regular,2016-03-18,old_client,...,2,F1,2025-07-01,0030,84,1510228,11552,1155,1,1
1,MX|0030|1510228,ENERGIZANTE,522512,5,VOLT PONCHE DE FRUTA LATA 473 ML 12,BODEGA Y PARTICULARES,"BODEGAS, TIENDAS",Regular,2016-03-18,old_client,...,2,F1,2025-07-01,0030,84,1510228,11552,1155,1,2
2,MX|0030|1510228,NECTAR,522632,5,PULP DURAZNO PET NO RETORNABLE 1000 ML 6,BODEGA Y PARTICULARES,"BODEGAS, TIENDAS",Regular,2016-03-18,old_client,...,2,F1,2025-07-01,0030,84,1510228,11552,1155,3,3
3,MX|0030|1510228,ISOTONICA,599587,5,SPORADE MORA PET NO RETORNABLE 1000 ML 6,BODEGA Y PARTICULARES,"BODEGAS, TIENDAS",Regular,2016-03-18,old_client,...,2,F1,2025-07-01,0030,84,1510228,11552,1155,9,4
4,MX|0030|1517017,AGUA,522795,5,VIDA AGUA PET NO RETORNABLE 1500 ML 12,BODEGA Y PARTICULARES,"BODEGAS, TIENDAS",Regular,2016-03-18,old_client,...,2,F1,2025-07-01,0030,84,1517017,11582,1158,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,MX|0030|1837592,AGUA,522352,1,VIDA AGUA PET NO RETORNABLE 1000 ML 12,NaN,"BODEGAS, TIENDAS",Irregular,2025-03-10,new_client,...,2,F1,2025-07-01,0030,84,1837592,11582,1158,1,1
463,MX|0030|1837593,AGUA,522353,1,VIDA AGUA PET NO RETORNABLE 625 ML 12,NaN,"BODEGAS, TIENDAS",Irregular,2025-03-10,new_client,...,2,F1,2025-07-01,0030,84,1837593,11582,1158,1,1
464,MX|0030|1837594,AGUA,522352,1,VIDA AGUA PET NO RETORNABLE 1000 ML 12,NaN,"BODEGAS, TIENDAS",Irregular,2025-03-10,new_client,...,2,F1,2025-07-01,0030,84,1837594,11582,1158,1,1
465,MX|0030|1837595,AGUA,522352,1,VIDA AGUA PET NO RETORNABLE 1000 ML 12,NaN,"BODEGAS, TIENDAS",Irregular,2025-03-10,new_client,...,2,F1,2025-07-01,0030,84,1837595,11582,1158,1,1


## 5.7 Filtro de Recomendaciones por Segmento de cliente

In [226]:
# Definir una función para filtrar las filas según el valor de 'new_segment'
def filtrar_segmento(group):
    segmento = group["new_segment"].iloc[
        0
    ]  # Obtener el valor del segmento para el grupo
    if segmento == "BLINDAR":
        return group.head(1)  # Si el segmento es 'Blindar', mantener la primera fila
    elif segmento == "MANTENER":
        return group.head(
            2
        )  # Si el segmento es 'Mantener', mantener las dos primeras filas
    elif segmento == "DESARROLLAR":
        return group.head(
            3
        )  # Si el segmento es 'Desarrollar', mantener las tres primeras filas
    elif segmento == "OPTIMIZAR":
        return group.head(
            4
        )  # Si el segmento es 'Optimizar', mantener las cuatro primeras filas
    else:
        return group  # Si el segmento no es ninguno de los especificados, mantener todas las filas

In [227]:
final_rec = (
    final_rec.groupby("id_cliente").apply(filtrar_segmento).reset_index(drop=True)
)

/tmp/ipykernel_14092/3325175710.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_rec.groupby("id_cliente").apply(filtrar_segmento).reset_index(drop=True)


In [228]:
final_rec.groupby("new_segment")["id_cliente"].nunique()

new_segment
BLINDAR        49
DESARROLLAR    60
MANTENER       19
OPTIMIZAR      57
Name: id_cliente, dtype: int64

## 5.8. Guardando recomendacion para D&A y Comercial

In [229]:
final_rec.id_cliente.nunique()

185

In [230]:
final_rec.dias_de_visita__c.value_counts(dropna=False)

dias_de_visita__c
2    315
Name: count, dtype: int64

In [231]:
final_rec.periodo_de_visita__c.value_counts(dropna=False)

periodo_de_visita__c
F1    315
Name: count, dtype: int64

In [232]:
final_rec.ultima_visita.value_counts(dropna=False)

ultima_visita
2025-07-01    315
Name: count, dtype: int64

### 5.8.1 Para D&A

In [233]:
fecha_tomorrow = (
    datetime.now(pytz.timezone("America/Lima")) + timedelta(days=1)
).strftime("%Y-%m-%d")
fecha_tomorrow

'2025-07-08'

In [234]:
final_rec.to_csv(
    f"Output/PS_piloto_data_v1/D_pan_recs_data_{fecha_tomorrow}.csv", index=False
)

In [235]:
# Establecer la conexión con S3
bucket_name = 'aje-analytics-ps-backup'  # nombre de bucket en S3
file_name = f'PS_Mexico/Output/PS_piloto_data_v1/D_pan_recs_data_{fecha_tomorrow}.csv'  # nombre para el archivo en S3
s3_path = f's3://{bucket_name}/{file_name}'

# Escribir el dataframe en S3 con AWS Data Wrangler
wr.s3.to_csv(final_rec, s3_path, index=False)

{'paths': ['s3://aje-analytics-ps-backup/PS_Mexico/Output/PS_piloto_data_v1/D_pan_recs_data_2025-07-08.csv'],
 'partitions_values': {}}

### 5.8.2 Para subir a PS

In [236]:
recomendaciones_para_salesforce = final_rec[
    ["cod_compania", "cod_sucursal", "cod_cliente", "cod_modulo", "sku"]
]
recomendaciones_para_salesforce["Pais"] = "MX"
recomendaciones_para_salesforce["Cajas"] = int(1)
recomendaciones_para_salesforce["Unidades"] = int(0)
recomendaciones_para_salesforce["Fecha"] = fecha_tomorrow
recomendaciones_para_salesforce = recomendaciones_para_salesforce[
    [
        "Pais",
        "cod_compania",
        "cod_sucursal",
        "cod_cliente",
        "cod_modulo",
        "sku",
        "Cajas",
        "Unidades",
        "Fecha",
    ]
]
recomendaciones_para_salesforce.columns = [
    "Pais",
    "Compania",
    "Sucursal",
    "Cliente",
    "Modulo",
    "Producto",
    "Cajas",
    "Unidades",
    "Fecha",
]

/tmp/ipykernel_14092/3636268013.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recomendaciones_para_salesforce["Pais"] = "MX"
/tmp/ipykernel_14092/3636268013.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recomendaciones_para_salesforce["Cajas"] = int(1)
/tmp/ipykernel_14092/3636268013.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [237]:
recomendaciones_para_salesforce["Compania"] = recomendaciones_para_salesforce[
    "Compania"
].apply(lambda x: str(int(x)).rjust(4, "0"))
recomendaciones_para_salesforce["Sucursal"] = recomendaciones_para_salesforce[
    "Sucursal"
].apply(lambda x: str(int(x)).rjust(2, "0"))

In [238]:
recomendaciones_para_salesforce.dtypes

Pais        object
Compania    object
Sucursal    object
Cliente     object
Modulo       int64
Producto     int64
Cajas        int64
Unidades     int64
Fecha       object
dtype: object

In [239]:
recomendaciones_para_salesforce.Sucursal.unique()

array(['84', '112'], dtype=object)

In [240]:
recomendaciones_para_salesforce.Compania.unique()

array(['0030'], dtype=object)

In [241]:
recomendaciones_para_salesforce.Cliente.nunique()

185

In [242]:
recomendaciones_para_salesforce

,Pais,Compania,Sucursal,Cliente,Modulo,Producto,Cajas,Unidades,Fecha
0,MX,0030,84,1510228,11552,522355,1,0,2025-07-08
1,MX,0030,84,1517017,11582,522795,1,0,2025-07-08
2,MX,0030,84,1524033,11582,517262,1,0,2025-07-08
3,MX,0030,112,1545235,10652,517262,1,0,2025-07-08
4,MX,0030,112,1545235,10652,800060,1,0,2025-07-08
...,...,...,...,...,...,...,...,...,...
310,MX,0030,84,1837592,11582,522352,1,0,2025-07-08
311,MX,0030,84,1837593,11582,522353,1,0,2025-07-08
312,MX,0030,84,1837594,11582,522352,1,0,2025-07-08
313,MX,0030,84,1837595,11582,522352,1,0,2025-07-08


In [243]:
recomendaciones_para_salesforce.Cliente.nunique()

185

In [244]:
rutas_cliente = df_ventas[["cod_cliente", "cod_ruta"]].drop_duplicates()

In [245]:
pd.merge(
    recomendaciones_para_salesforce,
    rutas_cliente,
    left_on="Cliente",
    right_on="cod_cliente",
    how="left",
).cod_ruta.nunique()

4

In [246]:
pd.merge(
    recomendaciones_para_salesforce,
    rutas_cliente,
    left_on="Cliente",
    right_on="cod_cliente",
    how="left",
).groupby("cod_ruta")["Cliente"].nunique()

cod_ruta
1065    37
1074    46
1155    42
1158    60
Name: Cliente, dtype: int64

In [247]:
pd.merge(
    recomendaciones_para_salesforce,
    rutas_cliente,
    left_on="Cliente",
    right_on="cod_cliente",
    how="left",
).groupby("cod_ruta")["Cliente"].nunique().mean()

46.25

**Conteo de marcas recomendadas**

In [248]:
count = (
    final_rec.groupby("id_cliente")["marca_rec"]
    .apply(lambda x: "-".join((x)))
    .reset_index(name="marcas_ordenadas")
)
count = (
    count.groupby("marcas_ordenadas")["id_cliente"]
    .nunique()
    .sort_values(ascending=False)
)
count.head(10)

marcas_ordenadas
ENERGIZANTE                   67
AGUA                          24
AGUA-ENERGIZANTE              12
ENERGIZANTE-CAFE              12
ENERGIZANTE-CAFE-ISOTONICA     6
AGUA-ISOTONICA                 5
AGUA-ENERGIZANTE-CAFE          4
ENERGIZANTE-ISOTONICA          4
ENERGIZANTE-CAFE-NECTAR        4
CAFE                           4
Name: id_cliente, dtype: int64

**Conteo de SKUs recomendadas**

In [249]:
count2 = (
    final_rec.groupby("id_cliente")["sku"]
    .apply(lambda x: "-".join(x.astype(str)))
    .reset_index(name="marcas_ordenadas")
)
count2 = (
    count2.groupby("marcas_ordenadas")["id_cliente"]
    .nunique()
    .sort_values(ascending=False)
    .reset_index()
)
count2["f"] = count2["id_cliente"] / count2["id_cliente"].sum()
count2["fa"] = count2["f"].cumsum()
count2

,marcas_ordenadas,id_cliente,f,fa
0,517262,28,0.151351,0.151351
1,517140,14,0.075676,0.227027
2,522352,9,0.048649,0.275676
3,599228,8,0.043243,0.318919
4,522355,8,0.043243,0.362162
...,...,...,...,...
88,522353-516209,1,0.005405,0.978378
89,522353-515614,1,0.005405,0.983784
90,522352-599587,1,0.005405,0.989189
91,522352-599212-622631,1,0.005405,0.994595


In [250]:
print(f"RESUMEN para {fecha_tomorrow}")
print(
    "Total de clientes a recomendar: ",
    recomendaciones_para_salesforce.Cliente.nunique(),
)
print("Combinaciones de recomendaciones de MARCAS:", count.shape[0])
print("Marcar recomendadas:", final_rec.marca_rec.unique())
print("Combinaciones de recomendaciones de SKUs:", count2.shape[0])
print(
    "Combinaciones de recomendaciones de SKUs al 80% de clientes:",
    count2[count2["fa"] <= 0.8].shape[0],
)
print("SKUs usados en la recomendacion:", final_rec["sku"].nunique())

RESUMEN para 2025-07-08
Total de clientes a recomendar:  185
Combinaciones de recomendaciones de MARCAS: 41
Marcar recomendadas: ['AGUA' 'ENERGIZANTE' 'CAFE' 'JUGOS LIGEROS' 'ISOTONICA'
 'SOPA INSTANTANEA' 'GASEOSAS' 'NECTAR' 'CONSERVA DE PESCADO' 'ATUN']
Combinaciones de recomendaciones de SKUs: 93
Combinaciones de recomendaciones de SKUs al 80% de clientes: 56
SKUs usados en la recomendacion: 33


In [251]:
recomendaciones_para_salesforce

,Pais,Compania,Sucursal,Cliente,Modulo,Producto,Cajas,Unidades,Fecha
0,MX,0030,84,1510228,11552,522355,1,0,2025-07-08
1,MX,0030,84,1517017,11582,522795,1,0,2025-07-08
2,MX,0030,84,1524033,11582,517262,1,0,2025-07-08
3,MX,0030,112,1545235,10652,517262,1,0,2025-07-08
4,MX,0030,112,1545235,10652,800060,1,0,2025-07-08
...,...,...,...,...,...,...,...,...,...
310,MX,0030,84,1837592,11582,522352,1,0,2025-07-08
311,MX,0030,84,1837593,11582,522353,1,0,2025-07-08
312,MX,0030,84,1837594,11582,522352,1,0,2025-07-08
313,MX,0030,84,1837595,11582,522352,1,0,2025-07-08


In [252]:
recomendaciones_para_salesforce.groupby("Cliente")["Producto"].nunique().mean()

1.7027027027027026

In [253]:
recomendaciones_para_salesforce.groupby("Sucursal")["Cliente"].nunique()

Sucursal
112     83
84     102
Name: Cliente, dtype: int64

In [254]:
asd = (
    recomendaciones_para_salesforce.groupby("Cliente")
    .agg({"Producto": ["count", "nunique"]})
    .reset_index()
)
asd.columns = ["Cliente", "count", "nunique"]
asd["dif"] = asd["count"] == asd["nunique"]
asd.sort_values(["dif", "count"])

,Cliente,count,nunique,dif
0,1510228,1,1,True
1,1517017,1,1,True
2,1524033,1,1,True
6,1545333,1,1,True
10,1545360,1,1,True
...,...,...,...,...
69,1600522,4,4,True
83,1638644,4,4,True
84,1638646,4,4,True
103,1739008,4,4,True


In [255]:
asd.dif.value_counts()

dif
True    185
Name: count, dtype: int64

In [256]:
recomendaciones_para_salesforce.to_csv(
    f"Output/PS_piloto_v1/D_base_pedidos_{fecha_tomorrow}.csv", index=False
)

In [257]:
# Establecer la conexión con S3
bucket_name = 'aje-analytics-ps-backup'  # nombre de bucket en S3
file_name = f'PS_Mexico/Output/PS_piloto_v1/D_base_pedidos_{fecha_tomorrow}.csv'  # nombre para el archivo en S3
s3_path = f's3://{bucket_name}/{file_name}'

# Escribir el dataframe en S3 con AWS Data Wrangler
wr.s3.to_csv(recomendaciones_para_salesforce, s3_path, index=False)

{'paths': ['s3://aje-analytics-ps-backup/PS_Mexico/Output/PS_piloto_v1/D_base_pedidos_2025-07-08.csv'],
 'partitions_values': {}}

In [258]:
# # Detener el monitoreo
# running = False
# monitor_thread.join()

# # Graficar consumo de RAM y CPU
# fig, ax1 = plt.subplots(figsize=(10, 5))

# ax1.set_xlabel("Tiempo (segundos)")
# ax1.set_ylabel("RAM usada (GB)", color="tab:blue")
# ax1.plot(timestamps, mem_usage_gb, marker="o", linestyle="-", color="tab:blue", label="RAM (GB)")
# ax1.tick_params(axis="y", labelcolor="tab:blue")

# ax2 = ax1.twinx()  # Crear segundo eje Y para CPU
# ax2.set_ylabel("CPU usada (%)", color="tab:red")
# ax2.plot(timestamps, cpu_usage_percent, marker="x", linestyle="--", color="tab:red", label="CPU (%)")
# ax2.tick_params(axis="y", labelcolor="tab:red")

# fig.suptitle("Consumo de CPU y RAM del Notebook")
# ax1.grid()
# fig.tight_layout()
# plt.show()

In [259]:
# # Apaga el Kernel para luego poder usar los demás notebooks sin llenar la memoria
# import IPython
# app = IPython.get_ipython()
# app.kernel.do_shutdown(False)  # True indica apagar completamente el kernel


In [260]:
# 100